# 🤖 Qwen3 MoE — Учебный Walkthrough

Добро пожаловать! Этот ноутбук проведёт вас через сборку Qwen3 MoE: от фундаментальных блоков до полной модели и мини‑демо генерации.

**Что вы сделаете:**
- 🧩 Соберёте модули: RMSNorm → RoPE → SwiGLU → GQA → Transformer → MoE
- 🧪 После каждой секции — быстрый чек форм и численной стабильности
- 🚀 Финал: `Qwen3MoEModel` + короткая генерация (greedy/sampling)


## ⚙️ Окружение и Reproducibility
Подготовьте импорты, выберите `device`, зафиксируйте сиды и проверьте доступность CUDA.

**🧪 Quick Check (кодовая ячейка):**

```python
import torch, random, numpy as np
def set_seed(s=42):
    random.seed(s); np.random.seed(s); torch.manual_seed(s); torch.cuda.manual_seed_all(s)
set_seed(42); print('CUDA:', torch.cuda.is_available())
```

## 🧭 Конфигурация модели

In [ ]:
"""
Qwen3 MoE Model Configuration

Определяет все гиперпараметры модели в одном месте.
"""
from dataclasses import dataclass
from typing import Optional


@dataclass
class Qwen3Config:
    """
    Конфигурация для Qwen3 MoE модели.

    Архитектура:
    ------------
    - Vocabulary: GPT-2 tokenizer (50257 токенов)
    - Embedding: 1024-dim continuous vectors
    - Transformer: 12 MoE blocks с GQA + RoPE + SwiGLU
    - MoE: 8 экспертов, 2 активных per token (25% активация)
    - Output: Language modeling head (1024 → 50257)

    Параметры:
    ----------
    Model Architecture:
        vocab_size: Размер словаря (GPT-2 = 50257)
        hidden_size: Размерность скрытого слоя (embedding dimension)
        num_layers: Количество MoE Transformer блоков
        intermediate_size: Размерность FFN внутри каждого эксперта

    Attention:
        num_attention_heads: Количество Query голов (для GQA)
        num_key_value_heads: Количество Key/Value голов (GQA группировка)
        max_position_embeddings: Максимальная длина последовательности
        rope_theta: Базовая частота для RoPE (10000.0 стандарт)

    MoE Specific:
        num_experts: Общее количество экспертов в каждом MoE слое
        top_k: Количество активных экспертов per token
        balance_loss_coef: Коэффициент для load balancing loss (обычно 0.01)

    Regularization:
        dropout: Dropout rate для регуляризации (0.0 = отключен)

    Training:
        initializer_range: Стандартное отклонение для инициализации весов

    Примеры:
    --------
    >>> # Конфигурация по умолчанию (0.6B параметров)
    >>> config = Qwen3Config()
    >>> print(f"Model size: ~{config.vocab_size * config.hidden_size / 1e9:.2f}B parameters")

    >>> # Кастомная конфигурация
    >>> config = Qwen3Config(
    ...     hidden_size=2048,
    ...     num_layers=24,
    ...     num_experts=16
    ... )
    """

    # Model Architecture
    vocab_size: int = 50257  # GPT-2 tokenizer
    hidden_size: int = 1024
    num_layers: int = 12
    intermediate_size: int = 2048  # 2 * hidden_size для каждого эксперта

    # Attention Configuration
    num_attention_heads: int = 16  # Query heads
    num_key_value_heads: int = 4   # KV heads (GQA: 4x группировка)
    max_position_embeddings: int = 2048
    rope_theta: float = 10000.0

    # MoE Configuration
    num_experts: int = 8
    top_k: int = 2
    balance_loss_coef: float = 0.01

    # Regularization
    dropout: float = 0.1

    # Training
    initializer_range: float = 0.02

    def __post_init__(self):
        """Валидация конфигурации после инициализации."""
        # Базовые проверки
        assert self.vocab_size > 0, "vocab_size должен быть положительным"
        assert self.hidden_size > 0, "hidden_size должен быть положительным"
        assert self.num_layers > 0, "num_layers должен быть положительным"
        assert self.intermediate_size > 0, "intermediate_size должен быть положительным"

        # Attention проверки
        assert self.num_attention_heads > 0, "num_attention_heads должен быть положительным"
        assert self.num_key_value_heads > 0, "num_key_value_heads должен быть положительным"
        assert (
            self.num_attention_heads % self.num_key_value_heads == 0
        ), "num_attention_heads должен делиться на num_key_value_heads"
        assert (
            self.hidden_size % self.num_attention_heads == 0
        ), "hidden_size должен делиться на num_attention_heads"

        # MoE проверки
        assert self.num_experts > 0, "num_experts должен быть положительным"
        assert self.top_k > 0, "top_k должен быть положительным"
        assert self.top_k <= self.num_experts, "top_k не может быть больше num_experts"

        # Regularization проверки
        assert 0.0 <= self.dropout <= 1.0, "dropout должен быть в диапазоне [0, 1]"

    def to_dict(self):
        """Конвертация конфигурации в словарь."""
        return {
            "vocab_size": self.vocab_size,
            "hidden_size": self.hidden_size,
            "num_layers": self.num_layers,
            "intermediate_size": self.intermediate_size,
            "num_attention_heads": self.num_attention_heads,
            "num_key_value_heads": self.num_key_value_heads,
            "max_position_embeddings": self.max_position_embeddings,
            "rope_theta": self.rope_theta,
            "num_experts": self.num_experts,
            "top_k": self.top_k,
            "balance_loss_coef": self.balance_loss_coef,
            "dropout": self.dropout,
            "initializer_range": self.initializer_range,
        }

max_position_embeddings = 2048


## 🧮 Нормализация: RMSNorm

In [ ]:
# Стандартная библиотека
from typing import Optional

# Сторонние библиотеки
import torch
import torch.nn as nn


class RMSNorm(nn.Module):
    """
    Description:
    ---------------
        Root Mean Square Layer Normalization — современная альтернатива LayerNorm.
        Формула: RMSNorm(x) = x / sqrt(mean(x²) + eps) * weight

        В отличие от LayerNorm, RMSNorm не центрирует данные (не вычитает среднее),
        что обеспечивает лучшую численную стабильность и производительность
        при обучении больших языковых моделей.

        Ключевые преимущества:
        - Меньше вычислений (нет центрирования)
        - Лучшая стабильность при больших моделях
        - Используется в современных архитектурах (LLaMA, Qwen, и др.)

    Args:
    ---------------
        normalized_shape: int или tuple размерностей для нормализации.
                          Обычно равен hidden_size модели.
        eps: Малая константа для численной устойчивости, предотвращает деление на ноль.
                          Добавляется под корень: sqrt(mean(x²) + eps).
        elementwise_affine: Если True, добавляет обучаемые параметры weight.
                            Если False, применяет только нормализацию без масштабирования.

    Examples:
    ---------------
        >>> import torch
        >>> rms_norm = RMSNorm(512)
        >>> x = torch.randn(10, 20, 512)
        >>> output = rms_norm(x)
        >>> output.shape
        torch.Size([10, 20, 512])

        >>> # Проверка нормализации: RMS должен быть близок к 1
        >>> rms_value = torch.sqrt(torch.mean(output**2, dim=-1))
        >>> print(f"RMS after normalization: {rms_value.mean():.4f}")
    """

    def __init__(
        self,
        normalized_shape: int,
        eps: float = 1e-6,
        elementwise_affine: bool = True
    ):
        super().__init__()

        # TODO: Сохраните normalized_shape для использования в forward
        # TODO: Сохраните eps для численной стабильности
        # TODO: Если elementwise_affine=True, создайте Parameter weight с формой (normalized_shape)
        # TODO: Инициализируйте weight единицами: torch.ones()
        # TODO: Если elementwise_affine=False, зарегистрируйте weight как None

        # Вопросы для размышления:
        # - Почему weight инициализируется единицами, а не нулями?
        # - Что произойдет, если eps слишком большой или слишком маленький?
        # - Зачем нужен параметр elementwise_affine?
        # pass

        self.normalized_shape = normalized_shape
        self.eps = eps

        # Создаем обучаемый параметр масштабирования (g в формуле RMSNorm)
        if elementwise_affine == True:
            # Инициализируем weight единицами для стабильности начального обучения
            # weight соответствует вектору g в формуле: g ⊙ (x/RMS(x))
            self.weight = nn.Parameter(
                torch.ones(normalized_shape)
            )
        else:
            # Если масштабирование не требуется, регистрируем пустой параметр
            # для совместимости с механизмами PyTorch (state_dict и др.)
            self.register_parameter('weight', None)
                    

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Description:
        ---------------
            Реализует формулу RMSNorm: x / sqrt(mean(x²) + eps) * weight.
            Нормализует входной тензор по его среднеквадратичному значению,
            без центрирования (в отличие от LayerNorm).

        Args:
        ---------------
            x: Входной тензор формы (..., normalized_shape)

        Returns:
        ---------------
            Нормализованный тензор той же формы

        Raises:
        ---------------
            RuntimeError: Если последняя размерность x не совпадает с normalized_shape
        """
        # TODO: Проверьте, что последняя размерность x равна self.normalized_shape
        # TODO: Вычислите квадраты элементов: x_squared = x * x или x.pow(2)
        # TODO: Вычислите среднее квадратов по последней оси: torch.mean
        # TODO: Вычислите RMS: torch.sqrt
        # TODO: Нормализуйте по формуле: RMSNorm(x) = x / sqrt(mean(x²) + eps)
        # TODO: Если есть weight, примените масштабирование: output = normalized * self.weight
        # TODO: Верните результат

        # Вопросы для размышления:
        # - Почему важно использовать keepdim=True при вычислении среднего?
        # - Как поведет себя функция на тензорах разной размерности?
        # - Что произойдет с градиентами при обратном проходе?
        # - Как RMSNorm влияет на распределение активаций?
        # pass

        # Проверяем соответствие размерности входного тензора
        if x.shape[-1] != self.normalized_shape:
            raise RuntimeError(
                f"Последняя размерность x должна быть равна {self.normalized_shape}"
            )

        # Вычисляем квадраты элементов
        x_sqr = x * x

        # Вычисляем среднее квадратов по последней оси
        # keepdim=True сохраняет размерность для корректного вещания при делении
        mean_sqr = torch.mean(x_sqr, dim=-1, keepdim=True)

        # Вычисляем RMS (корень из среднего квадратов) с добавлением eps для стабильности
        rms = torch.sqrt(mean_sqr + self.eps)

        # Нормализуем входной тензор, деля на RMS
        x_norm = x / rms

        # Применяем масштабирование, если есть weight
        if self.weight is not None:
            # Поэлементное умножение на обучаемый параметр weight
            return x_norm * self.weight
        
        return x_norm
        

    def extra_repr(self) -> str:
        """Строковое представление модуля для отладки."""
        return f'normalized_shape={self.normalized_shape}, eps={self.eps}, elementwise_affine={self.weight is not None}'

## 🧭 Позиционное кодирование: RoPE

In [ ]:
# Стандартная библиотека
from typing import Optional, Tuple, Union

# Сторонние библиотеки
import torch
import torch.nn as nn
import math


class RoPE(nn.Module):
    """
    Description:
    ---------------
        Rotary Position Embedding (RoPE) — метод позиционного кодирования,
        основанный на вращении векторов в комплексной плоскости.
        
        В отличие от абсолютных позиционных эмбеддингов, RoPE кодирует
        относительные позиции, что делает его особенно эффективным для
        моделей с длинным контекстом.
        
        Ключевые преимущества:
        - Инвариантность к сдвигу (shift invariance)
        - Эффективность вычислений
        - Хорошая экстраполяция на длины, превышающие обучающие
        - Совместимость с линейными attention механизмами
        
    Args:
    ---------------
        dim: Размерность эмбеддинга (должна быть четной для комплексного представления)
        base: База для вычисления частот (обычно 10000.0)
        max_position: Максимальная позиция для предварительного вычисления (кэширования)
        scale: Масштабирующий коэффициент для частот (используется для RoPE scaling)
        
    Examples:
    ---------------
        >>> import torch
        >>> rope = RoPE(dim=128)
        >>> q = torch.randn(2, 4, 128)  # [batch_size, seq_len, dim]
        >>> k = torch.randn(2, 4, 128)  # [batch_size, seq_len, dim]
        >>> q_pos, k_pos = rope(q, k)
        >>> q_pos.shape, k_pos.shape
        (torch.Size([2, 4, 128]), torch.Size([2, 4, 128]))
    """

    def __init__(
        self,
        dim: int,
        base: float = 10000.0,
        max_position: int = 2048,
        scale: float = 1.0
    ):
        super().__init__()
        
        # TODO: Проверьте, что dim четное (необходимо для комплексного представления)
        # TODO: Сохраните параметры (dim, base, max_position, scale)
        # TODO: Предварительно вычислите sin/cos таблицу частот для позиций [0, max_position)
        #       - Создайте тензор позиций;
        #       - Создайте тензор частот;
        #       - Вычислите углы для каждой позиции и частоты;
        #       - Вычислите sin и cos;
        #       - Сохраните кэш как буферы (не параметры).
        
        # Вопросы для размышления:
        # - Почему RoPE использует комплексное представление для кодирования позиций?
        # - Как выбор base влияет на частотные характеристики позиционного кодирования?
        # - Почему для длинных контекстов часто используют scale < 1.0?
        # pass

        if dim % 2 != 0:
            raise ValueError('Для корректной работы параметр dim должен быть четным')
        
        # Сохраняем основные параметры для использования в других методах
        self.dim = dim                    # Размерность эмбеддинга (должна быть четной)
        self.base = base                  # База для вычисления частот (обычно 10000.0)
        self.max_position = max_position  # Максимальная позиция для кэширования
        self.scale = scale                # Масштабирующий коэффициент для длинных контекстов

        # Создаем тензор позиций [0, 1, 2, ..., max_position-1]
        position = torch.arange(start = 0, end = max_position, step =1).float()

        # Создаем тензор с четными индексами [0, 2, 4, ...] для адресации пар измерений
        # Каждая пара (2i, 2i+1) будет обрабатываться как комплексное число
        idx = torch.arange(start=0, end=dim, step=2).float()

        # Вычисляем частоты для каждой пары измерений по формуле: ω_d = base^(-2d/D)
        # - Низкие частоты (начало тензора) меняются медленно с изменением позиции
        # - Высокие частоты (конец тензора) меняются быстрее
        # - scale < 1.0 замедляет вращение для лучшей экстраполяции на длинные контексты
        freqs = base ** (-idx / dim)

        # Вычисляем углы для каждой комбинации позиции и частоты
        # Применяем scale для поддержки длинных контекстов
        # Форма: (max_position, dim/2)
        angles = position.unsqueeze(1) * freqs.unsqueeze(0) / scale

        # Вычисляем sin и cos для каждого угла
        cos = torch.cos(angles)  # (max_position, dim/2)
        sin = torch.sin(angles)  # (max_position, dim/2)

        # Сохраняем sin и cos как буферы (не параметры)
        # Используем register_buffer для правильной работы с CUDA и сохранения/загрузки модели
        self.register_buffer('sin_cached', sin)
        self.register_buffer('cos_cached', cos)

        
    def _compute_rope_embeddings(
        self,
        x: torch.Tensor,
        positions: torch.Tensor,
        is_query: bool = True
    ) -> torch.Tensor:
        """
        Description:
        ---------------
            Применяет ротационное позиционное кодирование к входному тензору.
        
        Args:
        ---------------
            x: Входной тензор формы (..., seq_len, dim)
            positions: Тензор позиций формы (..., seq_len)
            is_query: Флаг, указывающий, является ли вход query (True) или key (False)
            
        Returns:
        ---------------
            Тензор с примененным позиционным кодированием той же формы, что и x
        """
        # TODO: Получите форму входного тензора x
        # TODO: Извлеките sin и cos для заданных позиций из кэша или вычислите их на лету
        # TODO: Если positions выходят за пределы max_position, вычислите sin и cos динамически
        # TODO: Примените вращение к каждой паре соседних измерений (dim[i], dim[i+1])
        #       - Для четных индексов i: x[..., i] = x[..., i] * cos - x[..., i+1] * sin
        #       - Для нечетных индексов i: x[..., i] = x[..., i] * sin + x[..., i-1] * cos
        # TODO: Если is_query=False (для ключей), инвертируйте направление вращения
        # TODO: Верните тензор с примененным позиционным кодированием
        
        # Вопросы для размышления:
        # - Почему для query и key используются разные направления вращения?
        # - Как RoPE обеспечивает относительное позиционное кодирование?
        # - Как работает экстраполяция на позиции за пределами max_position?
        # pass

        # Получаем форму входного тензора
        x_shape = x.shape

        if positions.max() < self.max_position:
            sin = self.sin_cached[positions]
            cos = self.cos_cached[positions]
        else:
            # Так же как в __init__
            idx = torch.arange(start=0, end=self.dim, step=2).float()
            freqs = self.base ** (-idx / self.dim)
            angles = positions.unsqueeze(1) * freqs.unsqueeze(0) / self.scale

            cos = torch.cos(angles)
            sin = torch.sin(angles)

        # Создаем выходной тензор той же формы, что и входной
        x_out = torch.zeros_like(x)

        # Применяем вращение в зависимости от типа входа (query или key)
        if is_query:
            # Для query - положительное направление вращения
            x_out[..., 0::2] = x[..., 0::2] * cos - x[..., 1::2] * sin
            x_out[..., 1::2] = x[..., 1::2] * cos + x[..., 0::2] * sin
        else:
            # Для key - отрицательное направление вращения
            x_out[..., 0::2] = x[..., 0::2] * cos + x[..., 1::2] * sin
            x_out[..., 1::2] = x[..., 1::2] * cos - x[..., 0::2] * sin

        return x_out

    def forward(
        self,
        query: torch.Tensor,
        key: torch.Tensor,
        positions: Optional[torch.Tensor] = None
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Description:
        ---------------
            Применяет RoPE к query и key тензорам.
        
        Args:
        ---------------
            query: Query тензор формы (..., seq_len, dim)
            key: Key тензор формы (..., seq_len, dim)
            positions: Опциональный тензор позиций. Если None, используется torch.arange(seq_len)
            
        Returns:
            Кортеж (query_pos, key_pos) с примененным позиционным кодированием
        """
        # TODO: Проверьте, что последнее измерение query и key равно self.dim
        # TODO: Получите seq_len из формы query
        # TODO: Если positions не указаны, создайте их через torch
        # TODO: Примените _compute_rope_embeddings к query
        # TODO: Примените _compute_rope_embeddings к key
        # TODO: Верните кортеж (query_pos, key_pos)
        
        # Вопросы для размышления:
        # - Как RoPE влияет на взаимодействие между query и key в attention?
        # - Почему важно применять RoPE к обоим тензорам - query и key?
        # - Как можно оптимизировать вычисления RoPE для больших моделей?
        # pass

        if query.shape[-1] != self.dim or key.shape[-1] != self.dim:
            raise ValueError(f'Размерность query и key должна быть равна {self.dim}')
        
        # Проверяем, что формы query и key совпадают по seq_len
        if query.shape[-2] != key.shape[-2]:
            raise ValueError(f'Длины последовательностей query и key должны совпадать')
        
        seq_len = query.shape[-2]

        if positions is not None:
            query_rope = self._compute_rope_embeddings(query, positions, is_query=True)
            key_rope   = self._compute_rope_embeddings(key,   positions, is_query=False)

            return query_rope, key_rope

        else:
            # Создаем позиции от 0 до seq_len-1 на том же устройстве, что и query
            positions = torch.arange(seq_len, device=query.device)
            # Применяем RoPE к query и key
            query_rope = self._compute_rope_embeddings(query, positions, is_query=True)
            key_rope = self._compute_rope_embeddings(key, positions, is_query=False)
            
            return query_rope, key_rope


    def extra_repr(self) -> str:
        """Строковое представление модуля для отладки."""
        return f'dim={self.dim}, base={self.base}, max_position={self.max_position}, scale={self.scale}'

## ⚡ Активации: SwiGLU

In [ ]:
# Стандартная библиотека
from typing import Optional

# Сторонние библиотеки
import torch
import torch.nn as nn
import torch.nn.functional as F


class Swish(nn.Module):
    """
    Description:
    ---------------
        Swish активация: x * sigmoid(x)
        
        Предложена в статье "Searching for Activation Functions" (Ramachandran et al., 2017).
        Также известна как SiLU (Sigmoid Linear Unit) в PyTorch.
        
        Формула: Swish(x) = x * sigmoid(x)
        
        Преимущества:
        - Гладкая функция (все производные существуют)
        - Не ограничена сверху (в отличие от sigmoid)
        - Имеет нелинейность, близкую к ReLU для положительных значений
        - Имеет небольшое подавление для отрицательных значений
        
    Args:
    ---------------
        beta: Опциональный параметр для масштабирования: x * sigmoid(beta * x)
              По умолчанию beta=1.0 (стандартный Swish)
        
    Returns:
    ---------------
        Тензор той же формы, что и вход, с примененной Swish активацией
        
    Examples:
    ---------------
        >>> import torch
        >>> swish = Swish()
        >>> x = torch.tensor([-2.0, -1.0, 0.0, 1.0, 2.0])
        >>> swish(x)
        tensor([-0.2384, -0.2689, 0.0000, 0.7311, 1.7616])
    """
    
    def __init__(self, beta: float = 1.0):
        super().__init__()
        # TODO: Сохраните beta параметр
        # pass

        self.beta = beta
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Description:
        ---------------
            Применяет Swish активацию к входному тензору.
        
        Args:
        ---------------
            x: Входной тензор любой формы
            
        Returns:
        ---------------
            Тензор той же формы с примененной Swish активацией
        """
        # TODO: Реализуйте Swish активацию: x * sigmoid(beta * x)
        # pass

        # Сигмоида масштабирует значения тензора от 0 до 1, 
        # затем умножаем на исходный тензор, тем самым сглаживая значения
        return x * torch.sigmoid(self.beta * x)


class SwiGLU(nn.Module):
    """
    Description:
    ---------------
        SwiGLU (Swish-Gated Linear Unit) - активационная функция,
        используемая в современных языковых моделях, включая Qwen3.
        
        Сочетает Swish активацию и механизм гейтинга (GLU).
        
        Формула:
        SwiGLU(x, W1, W2, b1, b2) = Swish(W1*x + b1) ⊙ (W2*x + b2)
        
        где:
        - W1, W2 - весовые матрицы
        - b1, b2 - векторы смещения (опциональные)
        - ⊙ - поэлементное умножение
        
        Преимущества:
        - Лучшая производительность по сравнению с ReLU/GELU в глубоких моделях
        - Эффективный механизм гейтинга для контроля потока информации
        - Используется в современных LLM (Qwen, PaLM, LLaMA)
        
    Args:
    ---------------
        input_dim: Размерность входного вектора
        output_dim: Размерность выходного вектора
        intermediate_dim: Размерность промежуточных матриц W1 и W2 (как правило 4*input_dim)
        bias: Использовать ли смещение в линейных преобразованиях (по умолчанию True)
        
    Returns:
    ---------------
        Тензор формы (..., output_dim) - результат применения SwiGLU
        
    Examples:
    ---------------
        >>> import torch
        >>> swiglu = SwiGLU(512, 512)
        >>> x = torch.randn(2, 10, 512)
        >>> output = swiglu(x)
        >>> output.shape
        torch.Size([2, 10, 512])
    """
    
    def __init__(
        self,
        input_dim: int,
        output_dim: int,
        intermediate_dim: Optional[int] = None,
        bias: bool = True
    ):
        super().__init__()
        
        # TODO: Если intermediate_dim не указан, установите его как 4*output_dim
        # TODO: Создайте линейный слой gate_proj для проекции входа в промежуточное представление
        # TODO: Создайте линейный слой value_proj для проекции входа в промежуточное представление
        # TODO: Создайте экземпляр Swish активации
        
        # Вопросы для размышления:
        # - Почему используется коэффициент 4 для промежуточной размерности?
        # - Какое преимущество дает механизм гейтинга по сравнению с простой активацией?
        # - Почему SwiGLU лучше работает в глубоких моделях по сравнению с ReLU/GELU?
        # pass

        self.input_dim  = input_dim
        self.output_dim = output_dim
        self.bias = bias

        self.swish = Swish()
        
        # Если intermediate_dim не указан, устанавливаем его как 4*input_dim
        if intermediate_dim is None:
            self.intermediate_dim = 4 * input_dim
        else:
            self.intermediate_dim = intermediate_dim

        # nn.Linear создает матрицу весов размера [intermediate_dim, input_dim] и вектор смещения [intermediate_dim]
        # Это соответствует проекции W1*x + b1 в формуле SwiGLU (расширяем)
        self.gate_proj = nn.Linear(self.input_dim, self.intermediate_dim, bias=self.bias)
        # Это соответствует проекции W2*x + b2 в формуле SwiGLU (расширяем)
        self.value_proj = nn.Linear(self.input_dim, self.intermediate_dim, bias=self.bias)
        # Проекция результата в выходную размерность (сжимаем)
        self.output_proj = nn.Linear(self.intermediate_dim, self.output_dim, bias=self.bias)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Description:
        ---------------
            Применяет SwiGLU активацию к входному тензору.
        
        Args:
        ---------------
            x: Входной тензор формы (..., input_dim)
            
        Returns:
        ---------------
            Тензор формы (..., output_dim) - результат применения SwiGLU
        """
        # TODO: Примените gate_proj к входу
        # TODO: Примените Swish активацию к результату gate_proj
        # TODO: Примените value_proj к входу
        # TODO: Перемножьте поэлементно результаты Swish(gate_proj(x)) и value_proj(x)
        # TODO: Верните результат
        
        # Вопросы для размышления:
        # - Как механизм гейтинга влияет на градиенты при обратном распространении?
        # - Почему важно использовать разные проекции для gate и value?
        # - Как SwiGLU способствует обучению более глубоких моделей?
        # pass

        # Создаем линейный слой, W1*x + b1
        gate_proj = self.gate_proj(x)
        # Создаем линейный слой, W2*x + b2
        value_proj = self.value_proj(x)
        # Применяем Swish активацию к результату gate_proj
        swish = self.swish.forward(gate_proj)

        # Поэлементное умножение результатов (механизм гейтинга)
        swiglu_intermediate = swish * value_proj
        
        # Проекция в выходную размерность (сжимаем: input == output)
        output = self.output_proj(swiglu_intermediate)
        
        return output

## 👀 Внимание: Grouped‑Query Attention (GQA)

In [ ]:
# Стандартная библиотека
from typing import Optional, Tuple, Union

# Сторонние библиотеки
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

# Локальные импорты
import sys
import os
parent_dir = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
sys.path.insert(0, parent_dir)
from positional_encoding.rope import RoPE


class GroupedQueryAttention(nn.Module):
    """
    Description:
    ---------------
        Grouped-Query Attention (GQA) - оптимизированная версия Multi-Head Attention,
        используемая в современных языковых моделях, включая Qwen3.
        
        В отличие от Multi-Head Attention, где каждая голова имеет свои проекции
        для query, key и value, в GQA запросы (queries) группируются, а ключи (keys)
        и значения (values) используются совместно несколькими головами внимания.
        
        Это позволяет сократить вычислительные затраты и память, сохраняя при этом
        выразительную мощность механизма внимания.
        
        Формула:
        GQA(Q, K, V) = Softmax(QK^T/√d_k)V
        
        где:
        - Q разделен на G групп (меньше, чем количество голов для ключей и значений)
        - K и V имеют H голов (H ≥ G)
        - Каждая группа запросов использует несколько голов ключей и значений
        
        Преимущества:
        - Снижение вычислительных затрат и использования памяти
        - Сохранение выразительной мощности механизма внимания
        - Улучшение масштабируемости для больших моделей
        
    Args:
    ---------------
        hidden_size: Размерность скрытого состояния == d_model (example: LLaMA 70B hidden_size = d_model = 8192)
        num_query_groups: Количество групп запросов
        num_attention_heads: Количество голов внимания (для ключей и значений)
        head_dim: Размерность каждой головы внимания
        dropout: Вероятность дропаута (по умолчанию 0.0)
        bias: Использовать ли смещение в линейных преобразованиях (по умолчанию True)
        use_rope: Использовать ли RoPE для позиционного кодирования (по умолчанию True)
        rope_theta: База для RoPE (по умолчанию 10000.0)
        rope_scaling: Масштабирование для RoPE (по умолчанию 1.0)
        max_position: Максимальная длина последовательности для RoPE (по умолчанию 2048)
        
    Returns:
    ---------------
        Тензор формы (batch_size, seq_len, hidden_size) - результат применения GQA
        
    Examples:
    ---------------
        >>> import torch
        >>> gqa = GroupedQueryAttention(
        ...     hidden_size=512,
        ...     num_query_groups=8,
        ...     num_attention_heads=16,
        ...     head_dim=64
        ... )
        >>> x = torch.randn(2, 10, 512)
        >>> output = gqa(x)
        >>> output.shape
        torch.Size([2, 10, 512])
    """
    
    def __init__(
        self,
        hidden_size: int,
        num_query_groups: int,
        num_attention_heads: int,
        head_dim: Optional[int] = None,
        dropout: float = 0.0,
        bias: bool = True,
        use_rope: bool = True,
        rope_theta: float = 10000.0,
        rope_scaling: float = 1.0,
        max_position: int = 2048
    ):
        super().__init__()
        
        # TODO: Проверьте, что num_attention_heads делится на num_query_groups
        # TODO: Проверьте, что hidden_size делится на num_attention_heads
        # TODO: Вычислите head_dim, если он не указан
        # TODO: Сохраните все параметры как атрибуты класса
        # TODO: Создайте проекции для query, key и value
        # TODO: Создайте проекцию для выхода
        # TODO: Создайте dropout слой
        # TODO: Если use_rope=True, создайте RoPE модуль
        
        # Вопросы для размышления:
        # - Почему GQA эффективнее, чем обычный Multi-Head Attention?
        # - Как количество групп запросов влияет на производительность и качество модели?
        # - Какие преимущества дает совместное использование ключей и значений?
        # - Как RoPE интегрируется с GQA?
        # pass

        assert num_attention_heads % num_query_groups == 0, "Количество голов внимания должно делиться на количество групп запросов"
        assert hidden_size % num_attention_heads == 0, "Размерность скрытого состояния должна делиться на количество голов внимания"
        
        # Делим скрытую размерность d_model (hidden_size) на число голов внимания h (num_attention_heads)
        # Каждая голова обрабатывает кусок размерности d_head = d_model / h, а конкатенация h голов возвращает исходную размерность
        head_dim = hidden_size // num_attention_heads if head_dim is None else head_dim

        self.hidden_size = hidden_size
        self.num_query_groups = num_query_groups
        self.num_attention_heads = num_attention_heads
        self.head_dim = head_dim
        self.dropout = dropout
        self.bias = bias
        self.use_rope = use_rope
        self.rope_theta = rope_theta
        self.rope_scaling = rope_scaling
        self.max_position = max_position

        # Проекции для query, key и value
        # В GQA: query использует группы, а key/value используют все головы
        self.query_proj = nn.Linear(in_features = hidden_size,
                                    out_features = num_query_groups * head_dim,
                                    bias=bias)

        self.key_proj   = nn.Linear(in_features = hidden_size,
                                    out_features = num_attention_heads * head_dim,
                                    bias=bias)

        self.value_proj = nn.Linear(in_features = hidden_size,
                                    out_features = num_attention_heads * head_dim,
                                    bias=bias)
        
        self.output_proj = nn.Linear(in_features = num_query_groups * head_dim,
                                    out_features = hidden_size,
                                    bias=bias)
        
        self.dropout = nn.Dropout(dropout)


        self.rope = RoPE(dim=head_dim) if use_rope else None

        
    def _split_heads(
        self, 
        x: torch.Tensor, 
        num_heads: int
    ) -> torch.Tensor:
        """
        Description:
        ---------------
            Разделяет последнюю размерность тензора на несколько голов внимания.
        
        Args:
        ---------------
            x: Входной тензор формы (batch_size, seq_len, hidden_size)
            num_heads: Количество голов внимания
            
        Returns:
        ---------------
            Тензор формы (batch_size, num_heads, seq_len, head_dim)
        """
        # TODO: Получите новую форму тензора
        # TODO: Измените форму тензора и транспонируйте размерности
        # TODO: Верните результат
        # pass

        # batch_size - количество последовательностей, обрабатываемых одновременно
        # seq_len - количество токенов в последовательности
        # total_dim - общая размерность (num_heads * head_dim)
        batch_size, seq_len, total_dim = x.shape

        # Вычисляем head_dim на основе total_dim и количества голов
        head_dim = total_dim // num_heads

        # .view() перестраивает тензор в новую форму, не изменяя данные.
        # Меняем форму с (batch, seq, total_dim) на (batch, seq, num_heads, head_dim)
        x = x.view(batch_size, seq_len, num_heads, head_dim)

        # Транспонируем для формата (batch, num_heads, seq, head_dim) согласно документации
        x = x.transpose(1, 2)

        return x

    def _repeat_kv_heads(self, kv: torch.Tensor) -> torch.Tensor:
        """
        Description:
        ---------------
            Повторяет key/value головы для соответствия количеству групп query.
            Это ключевая особенность GQA - каждая группа query использует
            несколько key/value голов.

        Args:
        ---------------
            kv: Тензор key или value формы (batch_size, num_attention_heads, seq_len, head_dim)

        Returns:
        ---------------
            Тензор формы (batch_size, num_query_groups, seq_len, head_dim)
            где key/value головы сгруппированы для каждой группы query
        """
        # TODO: Получите размерности входного тензора
        # TODO: Вычислите количество key/value голов на одну группу query
        # TODO: Измените форму тензора для группировки голов
        # TODO: Усредните головы внутри каждой группы
        # TODO: Верните результат

        # Вопросы для размышления:
        # - Почему мы группируем key/value головы, а не дублируем их?
        # - Как группировка влияет на выразительную способность модели?
        # - Какие альтернативы усреднению можно использовать (concatenation, max pooling)?
        # - Как соотношение num_attention_heads к num_query_groups влияет на эффективность?

        batch_size, num_kv_heads, seq_len, head_dim = kv.shape

        # Вычисляем, сколько key/value голов приходится на одну группу query
        heads_per_group = num_kv_heads // self.num_query_groups

        # Изменяем форму для группировки голов: (batch, heads, seq, dim) -> (batch, groups, heads_per_group, seq, dim)
        kv = kv.view(batch_size, self.num_query_groups, heads_per_group, seq_len, head_dim)

        # Усредняем головы внутри каждой группы по оси heads_per_group (индекс 2)
        kv = kv.mean(dim=2)  # Размерность: (batch_size, num_query_groups, seq_len, head_dim)

        return kv

    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        past_key_value: Optional[Tuple[torch.Tensor, torch.Tensor]] = None,
        output_attentions: bool = False,
        use_cache: bool = False
    ) -> Union[
        Tuple[torch.Tensor, Optional[Tuple[torch.Tensor, torch.Tensor]], Optional[torch.Tensor]],
        torch.Tensor
    ]:
        """
        Description:
        ---------------
            Применяет Grouped-Query Attention к входному тензору.
        
        Args:
        ---------------
            hidden_states: Входной тензор формы (batch_size, seq_len, hidden_size)
            attention_mask: Маска внимания (опционально)
            position_ids: Позиционные индексы для RoPE (опционально)
            past_key_value: Кэшированные ключи и значения (опционально)
            output_attentions: Возвращать ли веса внимания (опционально)
            use_cache: Использовать ли кэширование ключей и значений (опционально)
            
        Returns:
        ---------------
            Тензор формы (batch_size, seq_len, hidden_size) - результат применения GQA
            Опционально: кэшированные ключи и значения, веса внимания
        """
        # TODO: Получите размерности входного тензора
        # TODO: Примените проекции для query, key и value
        # TODO: Разделите query на группы, а key и value на головы
        # TODO: Если use_rope=True, примените RoPE к query и key
        # TODO: Если past_key_value не None, объедините с текущими key и value
        # TODO: Если use_cache=True, подготовьте новый past_key_value
        # TODO: Вычислите скалярное произведение query и key
        # TODO: Масштабируйте скалярное произведение
        # TODO: Если attention_mask не None, примените маску
        # TODO: Примените softmax к весам внимания
        # TODO: Примените dropout к весам внимания
        # TODO: Вычислите взвешенную сумму значений
        # TODO: Объедините головы внимания
        # TODO: Примените выходную проекцию
        # TODO: Верните результат и опциональные выходы
        
        # Вопросы для размышления:
        # - Как attention_mask влияет на веса внимания?
        # - Как кэширование ключей и значений ускоряет генерацию?
        # - Какие преимущества дает использование RoPE в GQA?
        # pass

        # Тензор скрытого состояния
        batch_size, seq_len, hidden_size = hidden_states.shape

        # Проекции для query, key и value
        # Используем фабрику линейных слоев для создания проекций
        query = self.query_proj(hidden_states)
        key   = self.key_proj(hidden_states)
        value = self.value_proj(hidden_states)

        # Разделяем query на группы, а key и value на головы
        query = self._split_heads(query, self.num_query_groups)
        key   = self._split_heads(key,   self.num_attention_heads)
        value = self._split_heads(value, self.num_attention_heads)

        # Применяем RoPE до группировки, когда тензоры в правильном формате
        if self.use_rope:
            if position_ids is None:
                # Создаем позиционные индексы для RoPE: [0, 1, 2, ..., seq_len-1]
                position_ids = torch.arange(seq_len, dtype=torch.long, device=hidden_states.device)

                # Трансформация 1D → 2D → Batch-compatible:
                # 1. unsqueeze(0): [0,1,2,3] → [[0,1,2,3]] (добавляем batch размерность)
                # 2. expand(batch_size, -1): [[0,1,2,3]] → [[0,1,2,3], [0,1,2,3], ...]
                # Результат: каждый элемент в batch получает одинаковые позиции
                # expand() эффективен - создает view без копирования данных
                position_ids = position_ids.unsqueeze(0).expand(batch_size, -1)

            # RoPE применяется к каждой голове отдельно
            # Сначала изменяем формат для применения RoPE
            # Используем reshape() вместо view() так как после transpose() тензор может быть не-contiguous
            query_for_rope = query.reshape(batch_size * self.num_query_groups, seq_len, self.head_dim)
            key_for_rope   = key.reshape(batch_size * self.num_attention_heads, seq_len, self.head_dim)

            # Расширяем position_ids для всех голов
            pos_query = position_ids.unsqueeze(1).expand(-1, self.num_query_groups, -1).contiguous().view(-1, seq_len)
            pos_key = position_ids.unsqueeze(1).expand(-1, self.num_attention_heads, -1).contiguous().view(-1, seq_len)

            # Применяем RoPE
            query_rope, _ = self.rope(query_for_rope, query_for_rope, pos_query)
            _, key_rope = self.rope(key_for_rope, key_for_rope, pos_key)

            # Возвращаем к исходному формату
            query = query_rope.view(batch_size, self.num_query_groups, seq_len, self.head_dim)
            key = key_rope.view(batch_size, self.num_attention_heads, seq_len, self.head_dim)

        # Применяем группировку key/value для соответствия query группам
        key   = self._repeat_kv_heads(key)
        value = self._repeat_kv_heads(value)

        # Все тензоры уже в формате (batch, heads, seq, dim) после _split_heads и _repeat_kv_heads
        # query: (batch_size, num_query_groups, seq_len, head_dim)
        # key:   (batch_size, num_query_groups, seq_len, head_dim)
        # value: (batch_size, num_query_groups, seq_len, head_dim)

        # Объединяем с past_key_value, если предоставлено
        # Формат: (batch, heads, seq, dim), конкатенируем по seq_len (dim=2)
        if past_key_value is not None:
            key = torch.cat([past_key_value[0], key], dim=2)
            value = torch.cat([past_key_value[1], value], dim=2)

        # Подготавливаем новый past_key_value
        if use_cache:
            past_key_value = (key, value)

        # Вычисляем скалярное произведение query и key и масштабируем один раз
        scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(self.head_dim)
        # Применяем маску внимания
        if attention_mask is not None:
            scores = scores + attention_mask
        # Применяем softmax к весам внимания
        weights = F.softmax(scores, dim=-1)
        # Применяем dropout к весам внимания
        weights = self.dropout(weights)
        # Вычисляем взвешенную сумму значений
        context = torch.matmul(weights, value)
        # Объединяем головы внимания: (batch, heads, seq, dim) -> (batch, seq, heads, dim)
        context = context.transpose(1, 2).contiguous()
        # Объединяем головы: (batch, seq, heads, dim) -> (batch, seq, heads*dim)
        context = context.view(batch_size, seq_len, self.num_query_groups * self.head_dim)
        # Применяем выходную проекцию
        output = self.output_proj(context)
        # Возвращаем результат в зависимости от флагов
        if use_cache and output_attentions:
            return output, past_key_value, weights
        elif use_cache:
            return output, past_key_value, None
        elif output_attentions:
            return output, None, weights
        else:
            return output


## 🧱 Базовый Transformer‑блок (без MoE)

In [ ]:
# Стандартная библиотека
from typing import Optional, Tuple, Union

# Сторонние библиотеки
import torch
import torch.nn as nn

# Локальные импорты
import sys
import os
parent_dir = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
sys.path.insert(0, parent_dir)
from normalization.rmsnorm import RMSNorm
from attention.gqa import GroupedQueryAttention
from activations.swiglu import SwiGLU


class TransformerBlock(nn.Module):
    """
    Description:
    ---------------
        Блок Transformer для архитектуры Qwen3 MoE. Использует Pre-Norm архитектуру
        с Grouped-Query Attention и SwiGLU активацией.

        Архитектура блока:
        Input → RMSNorm → GQA → Residual → RMSNorm → SwiGLU → Residual → Output

    Args:
    ---------------
        hidden_size: Размерность скрытого состояния (d_model)
        num_query_groups: Количество групп запросов для GQA
        num_attention_heads: Количество голов внимания для key/value
        intermediate_size: Размерность промежуточного слоя в SwiGLU (по умолчанию 4 * hidden_size)
    """

    def __init__(
        self,
        hidden_size: int,
        num_query_groups: int,
        num_attention_heads: int,
        intermediate_size: Optional[int] = None
    ):
        super().__init__()
        # TODO: Проверьте валидность параметров
        # TODO: Вычислите intermediate_size если не указан (4 * hidden_size)
        # TODO: Сохраните параметры как атрибуты
        # TODO: Создайте self.attention_norm = RMSNorm(hidden_size)
        # TODO: Создайте self.attention = GroupedQueryAttention(...)
        # TODO: Создайте self.ffn_norm = RMSNorm(hidden_size)
        # TODO: Создайте self.feed_forward = SwiGLU(...)

        # --- Валидация параметров -------------------------------------------------
        assert (
            isinstance(hidden_size, int) and hidden_size > 0
        ), "hidden_size должен быть положительным целым числом"
        assert (
            isinstance(num_query_groups, int) and num_query_groups > 0
        ), "num_query_groups должен быть положительным целым числом"
        assert (
            isinstance(num_attention_heads, int) and num_attention_heads > 0
        ), "num_attention_heads должен быть положительным целым числом"

        # Ключевая проверка для GQA архитектуры:
        assert (
            num_attention_heads % num_query_groups == 0
        ), (
            "num_attention_heads должен делиться на num_query_groups для "
            "корректной работы GQA"
        )
        # Проверка делимости hidden_size на число голов
        # Тензор скрытого состояния должен равномерно делиться на число голов
        assert (
            hidden_size % num_attention_heads == 0
        ), "hidden_size должен делиться на num_attention_heads"

        # Проверка intermediate_size если указан
        if intermediate_size is not None:
            assert (
                isinstance(intermediate_size, int) and intermediate_size > 0
            ), "intermediate_size должен быть положительным целым числом"

        # --- Инициализация атрибутов ----------------------------------------------
        self.hidden_size = hidden_size
        self.num_query_groups = num_query_groups
        self.num_attention_heads = num_attention_heads
        self.intermediate_size = (
            intermediate_size if intermediate_size is not None else 4 * hidden_size
        )

        # --- Компоненты нормализации и подблоков ----------------------------------
        self.attention_norm = RMSNorm(hidden_size)
        self.attention = GroupedQueryAttention(
            hidden_size=hidden_size,
            num_query_groups=num_query_groups,
            num_attention_heads=num_attention_heads,
        )
        self.ffn_norm = RMSNorm(hidden_size)
        self.feed_forward = SwiGLU(
            input_dim=self.hidden_size,
            output_dim=self.hidden_size,
            intermediate_dim=self.intermediate_size,
        )
        
        
    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        past_key_value: Optional[Tuple[torch.Tensor, torch.Tensor]] = None,
        output_attentions: bool = False,
        use_cache: bool = False
    ) -> Union[torch.Tensor, Tuple]:
        """
        Description:
        ---------------
            Применяет Transformer блок к входному тензору.
            Input → RMSNorm → GQA → Residual → RMSNorm → SwiGLU → Residual → Output

        Args:
        ---------------
            hidden_states: Входной тензор формы (batch_size, seq_len, hidden_size)

        Returns:
        ---------------
            Тензор формы (batch_size, seq_len, hidden_size) - выход Transformer блока
        """
        # TODO: Сохраните входной тензор для первого residual connection
        # TODO: Примените attention_norm
        # TODO: Примените self.attention
        # TODO: Добавьте первый residual connection
        # TODO: Сохраните результат для второго residual connection
        # TODO: Примените ffn_norm
        # TODO: Примените self.feed_forward
        # TODO: Добавьте второй residual connection
        # TODO: Верните результат

        # Вопросы для размышления:
        # - Почему нормализация применяется ДО attention/ffn, а не после?
        # - Как residual connections помогают при обучении глубоких сетей?

        # ──────────────────────────────────────────────────────────────────────────
        # ПЕРВЫЙ RESIDUAL BLOCK: Self-Attention (GQA)
        # ──────────────────────────────────────────────────────────────────────────

        # Сохраняем вход для остаточной связи (residual).
        residual_1 = hidden_states

        # Преднормализация улучшает устойчивость и качество внимания.
        normed = self.attention_norm(hidden_states)

        # Вызываем модуль группового внимания. Он может вернуть:
        # - только выход (Tensor), либо
        # - кортеж (att_output, present_key_value, attn_weights).
        att_output = self.attention(
            hidden_states=normed,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_value=past_key_value,
            output_attentions=output_attentions,
            use_cache=use_cache,
        )

        if isinstance(att_output, tuple):
            att_output, present_key_value, attn_weights = att_output
        else:
            present_key_value = None
            attn_weights = None

        # Первая residual-связь: складываем вход и выход подблока.
        hidden_states = att_output + residual_1

        # ──────────────────────────────────────────────────────────────────────────
        # ВТОРОЙ RESIDUAL BLOCK: Feed-Forward (SwiGLU)
        # ──────────────────────────────────────────────────────────────────────────

        residual_2 = hidden_states

        # Преднормализация перед FFN по тем же причинам, что и перед вниманием.
        normed = self.ffn_norm(hidden_states)

        # Применяем нелинейную проекцию SwiGLU с расширением размерности
        # до intermediate_size и обратной проекцией к hidden_size.
        ffn_output = self.feed_forward(normed)

        # Вторая residual-связь.
        hidden_states = ffn_output + residual_2

        if use_cache or output_attentions:
            return hidden_states, present_key_value, attn_weights

        return hidden_states





## 🛣️ MoE: Router

In [ ]:
# Стандартная библиотека
import math
from typing import Tuple, Optional

# Сторонние библиотеки
import torch
import torch.nn as nn
import torch.nn.functional as F


class MoERouter(nn.Module):
    """
    Description:
    ---------------
        MoE Router (Mixture-of-Experts Router) для архитектуры Qwen3.

        Роутер решает для каждого токена:
        1. Какие K экспертов из N активировать (Top-K selection)
        2. С какими весами комбинировать их выходы (gating weights)
        3. Как балансировать нагрузку между экспертами (load balancing)

        Архитектура:
        Input Token → Linear Projection → Softmax → Top-K Selection → Gating Weights

        Для этой модели (0.6B): N=8 экспертов, K=2 активных per token
        Для справки, Qwen3-30B использует: N=128 экспертов, K=8 активных per token

    Mathematical Formulation:
    ---------------
        1. Gating scores: g = Softmax(W_g * x)
           где W_g - обучаемая матрица размера (hidden_size, num_experts)

        2. Top-K selection: indices, weights = TopK(g, k=top_k)
           Выбираем K экспертов с наибольшими весами

        3. Renormalization: weights = Softmax(weights)
           Нормализуем веса выбранных экспертов (сумма = 1)

        4. Load balancing loss: L_balance = α * mean(f * P)
           где f - частота выбора эксперта, P - средний вес эксперта

    Args:
    ---------------
        hidden_size: Размерность входного скрытого состояния
        num_experts: Общее количество экспертов (N)
        top_k: Количество активных экспертов per token (K)
        capacity_factor: Фактор емкости для ограничения токенов per expert (default: 1.25)
        balance_loss_coef: Коэффициент для load balancing loss (default: 0.01)

    Returns (from forward):
    ---------------
        routing_weights: Тензор формы (batch_size, seq_len, top_k)
                        Веса для каждого из K выбранных экспертов
        selected_experts: Тензор формы (batch_size, seq_len, top_k) dtype=long
                         Индексы выбранных экспертов [0, num_experts)
        balance_loss: Скаляр - loss для балансировки нагрузки между экспертами

    Example:
    ---------------
        >>> # Для модели 0.6B
        >>> router = MoERouter(hidden_size=512, num_experts=8, top_k=2)
        >>> x = torch.randn(2, 10, 512)  # (batch=2, seq=10, hidden=512)
        >>> weights, experts, loss = router(x)
        >>> weights.shape  # torch.Size([2, 10, 2])
        >>> experts.shape  # torch.Size([2, 10, 2])
        >>> loss.item()    # Скаляр loss
    """

    def __init__(
        self,
        hidden_size: int,
        num_experts: int,
        top_k: int = 2,
        capacity_factor: float = 1.25,
        balance_loss_coef: float = 0.01
    ):
        super().__init__()

        # TODO: Проверьте валидность параметров (hidden_size, num_experts, top_k)
        # TODO: Убедитесь что top_k <= num_experts
        # TODO: Сохраните все параметры как атрибуты класса
        # TODO: Создайте self.gate - линейный слой для проекции в пространство экспертов
        #       Размеры: (hidden_size) -> (num_experts)
        # TODO: Инициализируйте веса gate небольшими значениями для стабильности

        # Вопросы для размышления:
        # - Почему важно, чтобы top_k был меньше num_experts?
        # - Как capacity_factor влияет на балансировку нагрузки?
        # - Зачем нужна небольшая инициализация весов gate?
        # - Какие альтернативы Softmax можно использовать для gating?
        # pass

        # --- Валидация параметров -------------------------------------------------
        # Используем assert для раннего обнаружения ошибок конфигурации.
        assert isinstance(hidden_size, int) and hidden_size > 0, (
            "hidden_size должен быть положительным целым числом"
        )
        assert isinstance(num_experts, int) and num_experts > 0, (
            "num_experts должен быть положительным целым числом"
        )
        assert isinstance(top_k, int) and top_k > 0, (
            "top_k должен быть положительным целым числом"
        )
        assert top_k <= num_experts, (
            "num_experts должено быть больше или равно top_k"
        )
        assert isinstance(capacity_factor, float) and capacity_factor > 0, (
            "capacity_factor должен быть положительным числом"
        )
        assert isinstance(balance_loss_coef, float) and balance_loss_coef >= 0, (
            "balance_loss_coef должен быть неотрицательным числом"
        )

        # --- Инициализация атрибутов ----------------------------------------------
        # Храним параметры как атрибуты экземпляра, чтобы использовать их
        # при дальнейшей маршрутизации и расчёте регуляризаций.
        self.hidden_size = hidden_size
        self.num_experts = num_experts
        self.top_k = top_k
        self.capacity_factor = capacity_factor
        self.balance_loss_coef = balance_loss_coef

        # Линейный слой-gate предсказывает логиты по экспертам на основе входного скрытого состояния
        # последующий softmax (как правило, в forward) превращает их в вероятности.
        self.gate = nn.Linear(hidden_size, num_experts)

        # Инициализация: небольшой нормальный шум ускоряет сходимость.
        self.gate.weight.data.normal_(0, 0.01)

        # Нулевой сдвиг предотвращает смещение распределения по экспертам на старте обучения
        # проверка на наличие bias — на случай future-refactor.
        if self.gate.bias is not None:
            self.gate.bias.data.zero_()


    def forward(
        self,
        hidden_states: torch.Tensor,
        training: bool = True
    ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """
        Description:
        ---------------
            Применяет MoE routing к входным скрытым состояниям.

            Процесс:
            1. Проекция входа в пространство экспертов
            2. Вычисление gating scores через Softmax
            3. Top-K selection - выбор K лучших экспертов
            4. Renormalization весов выбранных экспертов
            5. Вычисление load balancing loss (только при training=True)

        Args:
        ---------------
            hidden_states: Входной тензор формы (batch_size, seq_len, hidden_size)
            training: Флаг режима обучения (для load balancing loss)

        Returns:
        ---------------
            routing_weights: Тензор формы (batch_size, seq_len, top_k)
                           Нормализованные веса для выбранных экспертов
            selected_experts: Тензор формы (batch_size, seq_len, top_k)
                            Индексы выбранных экспертов
            balance_loss: Скаляр - load balancing loss (0.0 если training=False)
        """
        # TODO: Получите размерности входного тензора (batch_size, seq_len, hidden_size)
        # TODO: Примените self.gate к hidden_states для получения логитов
        # TODO: Примените Softmax по оси num_experts для получения gating_scores
        #       Это дает распределение вероятностей по всем экспертам
        # TODO: Используйте torch.topk для выбора top_k экспертов
        #       Получите: routing_weights (веса), selected_experts (индексы)
        # TODO: Ре-нормализуйте routing_weights через Softmax
        #       Важно: веса K выбранных экспертов должны суммироваться в 1
        # TODO: Если training=True, вычислите load balancing loss
        #       Используйте вспомогательный метод _compute_balance_loss
        # TODO: Верните (routing_weights, selected_experts, balance_loss)

        # Вопросы для размышления:
        # - Почему нужна ре-нормализация после Top-K selection?
        # - Что произойдет, если все токены выберут одних и тех же экспертов?
        # - Как Top-K selection влияет на вычислительную эффективность?
        # - Почему balance_loss вычисляется только при training=True?
        # pass

        batch_size, seq_len, hidden_size = hidden_states.shape

        # Linear projection (W·x + b)
        # Проекция токенов в пространство экспертов: (B, S, H) → (B, S, N)
        # Логиты для всех N экспертов (например, 8 для модели 0.6B)
        logits = self.gate(hidden_states)

        # Softmax по оси num_experts для получения gating_scores
        # Распределение вероятностей по всем экспертам
        gating_scores = F.softmax(
            input = logits,
            dim = -1
        )
        
        # Top-K selection - выбор K лучших экспертов
        # Получение routing_weights (веса) и selected_experts (индексы)
        routing_weights, selected_experts = torch.topk(
            input = gating_scores,
            k = self.top_k,
            dim = -1
        )

        # Renormalization весов выбранных экспертов
        # Нормализация весов K выбранных экспертов (сумма = 1)
        routing_weights = F.softmax(
            input = routing_weights,
            dim = -1
        )

        if training:
            load_balance_loss = self._compute_balance_loss(
                gating_scores = gating_scores,
                selected_experts = selected_experts
            )
        else:
            load_balance_loss = torch.tensor(0.0, device=gating_scores.device)

        return routing_weights, selected_experts, load_balance_loss


    def _compute_balance_loss(
        self,
        gating_scores: torch.Tensor,
        selected_experts: torch.Tensor
    ) -> torch.Tensor:
        """
        Description:
        ---------------
            Вычисляет load balancing loss для равномерного распределения нагрузки.

            Цель: Предотвратить ситуацию, когда модель использует только малую часть экспертов.

            Формула: L_balance = α * num_experts * Σ(f_i * P_i)
            где:
            - f_i - fraction of tokens routed to expert i
            - P_i - mean gating score for expert i
            - α - balance_loss_coef

        Args:
        ---------------
            gating_scores: Тензор формы (batch_size, seq_len, num_experts)
                          Softmax scores для всех экспертов
            selected_experts: Тензор формы (batch_size, seq_len, top_k)
                            Индексы выбранных экспертов

        Returns:
        ---------------
            balance_loss: Скаляр тензор - loss для балансировки
        """
        # TODO: Вычислите frequency (f_i) - сколько токенов выбрали каждого эксперта
        #       Подсказка: используйте torch.bincount или создайте one-hot и усредните
        # TODO: Вычислите mean gating probability (P_i) для каждого эксперта
        #       Подсказка: усредните gating_scores по batch и sequence dimensions
        # TODO: Вычислите loss = balance_loss_coef * num_experts * sum(f_i * P_i)
        # TODO: Верните balance_loss

        # Вопросы для размышления:
        # - Почему мы умножаем на num_experts в формуле?
        # - Как этот loss влияет на распределение нагрузки?
        # - Что произойдет, если balance_loss_coef слишком большой?
        # - Какие альтернативные метрики балансировки существуют?
        # pass

        if gating_scores.dim() != 3:
            raise ValueError("gating_scores должен иметь форму (B, S, N).")
        if selected_experts.dim() != 3:
            raise ValueError("selected_experts должен иметь форму (B, S, K).")
        if gating_scores.size(-1) != self.num_experts:
            raise ValueError("Последняя размерность gating_scores должна быть N.")

        # # .view() перестраивает тензор уже выбранных экспертов в новую форму, не изменяя данные.
        # Было: (batch_size, seq_len, top_k) = (2, 10, 8)
        # Стало: (160,) — все индексы в одном массиве, мы получаем один длинный одномерный вектор
        flattened_experts = selected_experts.view(-1)

        # expert_counts[i] = сколько раз эксперт i был выбран
        expert_counts = torch.bincount(
            flattened_experts,
            minlength=self.num_experts  # Гарантируем вектор длины num_experts (например, 8)
        )

        # Общее количество выборов = batch_size * seq_len * top_k
        batch_size, seq_len, top_k = selected_experts.shape
        total_selections = batch_size * seq_len * top_k

        # f_i = (количество раз, когда эксперт i был выбран) / (общее количество выборов)
        f_i = expert_counts.float() / total_selections

        # Зачем вычисляем среднее для gating_scores, когда это уже тензор вероятностей экспертов после softmax?
        # Потому что нам нужно знать среднюю уверенность модели в каждом эксперте по всем токенам. Это отличается от частоты выбора:
        #   - f_i = как часто эксперт попадает в Top-K (0 или 1 для каждого токена)
        #   - P_i = какую среднюю вероятность модель назначает эксперту (до Top-K)
        # Произведение f_i * P_i максимально, когда эксперт и часто выбирается, и модель в нём уверена → это дисбаланс → высокий loss → градиент штрафует.
        P_i = gating_scores.mean(dim=(0, 1))

        balance_loss = self.balance_loss_coef * self.num_experts * (f_i * P_i).sum()

        return balance_loss


    def expert_capacity(self, num_tokens: int) -> int:
        """
        Description:
        ---------------
            Вычисляет максимальную емкость каждого эксперта.

            Capacity = (num_tokens / num_experts) * capacity_factor * top_k

            Это ограничивает количество токенов, которые может обработать один эксперт,
            предотвращая перегрузку отдельных экспертов.

        Args:
        ---------------
            num_tokens: Общее количество токенов (batch_size * seq_len)

        Returns:
        ---------------
            capacity: Максимальное количество токенов per expert
        """
        # TODO: Вычислите базовую capacity = num_tokens / num_experts
        # TODO: Умножьте на capacity_factor для запаса
        # TODO: Умножьте на top_k (каждый токен идет к K экспертам)
        # TODO: Округлите до целого числа (ceil)
        # TODO: Верните capacity

        # Вопросы для размышления:
        # - Зачем нужен capacity_factor > 1.0?
        # - Что делать с токенами, превышающими capacity?
        # - Как capacity влияет на memory footprint?
        # pass

        capacity = math.ceil((num_tokens / self.num_experts) * self.capacity_factor * self.top_k)

        return capacity

## 🧑‍🏫 MoE: Expert

In [ ]:
# Стандартная библиотека
from typing import Optional

# Сторонние библиотеки
import torch
import torch.nn as nn

# Локальные импорты
from experiments.domain.activations.swiglu import SwiGLU


class Expert(nn.Module):
    """
    Description:
    ---------------
        Expert Network для Mixture-of-Experts архитектуры.

        Каждый эксперт - это независимая feed-forward сеть, которая обрабатывает
        токены, направленные к ней Router'ом.

        Архитектура:
        Input (hidden_size) → SwiGLU FFN → Output (hidden_size)

        Внутри SwiGLU:
        hidden_size → intermediate_size (с gating) → hidden_size

        Для модели 0.6B:
        - hidden_size = 512
        - intermediate_size = 2048 (обычно 4 * hidden_size)
        - num_experts = 8 (каждый с независимыми весами)

    Mathematical Flow:
    ---------------
        x ∈ ℝ^(batch×seq×hidden)
            ↓
        SwiGLU(x) = Swish(W1·x) ⊙ (W2·x)  [intermediate_dim]
            ↓
        W3·SwiGLU(x) + b3
            ↓
        output ∈ ℝ^(batch×seq×hidden)

    Args:
    ---------------
        hidden_size: Размерность входа и выхода (должна совпадать с hidden_size модели)
        intermediate_size: Размерность промежуточного слоя (обычно 4 * hidden_size)
        dropout: Dropout вероятность для регуляризации (default: 0.0)

    Returns (from forward):
    ---------------
        output: Тензор формы (batch_size, seq_len, hidden_size)
                Преобразованные скрытые состояния

    Example:
    ---------------
        >>> # Создание одного эксперта для модели 0.6B
        >>> expert = Expert(hidden_size=512, intermediate_size=2048)
        >>> x = torch.randn(2, 10, 512)  # (batch=2, seq=10, hidden=512)
        >>> output = expert(x)
        >>> output.shape  # torch.Size([2, 10, 512])

        >>> # Создание нескольких экспертов
        >>> num_experts = 8
        >>> experts = nn.ModuleList([
        ...     Expert(hidden_size=512, intermediate_size=2048)
        ...     for _ in range(num_experts)
        ... ])
    """

    def __init__(
        self,
        hidden_size: int,
        intermediate_size: int,
        dropout: float = 0.0
    ):
        super().__init__()

        # TODO: Проверьте валидность параметров
        #       - hidden_size должен быть положительным целым числом
        #       - intermediate_size должен быть положительным целым числом
        #       - dropout должен быть в диапазоне [0.0, 1.0)
        # TODO: Сохраните параметры как атрибуты класса
        # TODO: Создайте self.ffn - экземпляр SwiGLU
        #       Параметры: input_dim=hidden_size, output_dim=hidden_size,
        #                  intermediate_dim=intermediate_size
        # TODO: Создайте self.dropout - слой Dropout с заданной вероятностью
        #       (даже если dropout=0.0, создайте слой для единообразия)

        # Вопросы для размышления:
        # - Почему intermediate_size обычно в 4 раза больше hidden_size?
        # - Зачем нужен dropout в экспертах?
        # - Как SwiGLU отличается от обычного ReLU FFN?
        # - Почему каждый эксперт должен иметь одинаковую архитектуру?
        # pass

        # --- Валидация параметров -------------------------------------------------
        # Используем assert для раннего обнаружения ошибок конфигурации.
        assert isinstance(hidden_size, int) and hidden_size > 0, (
            "hidden_size должен быть положительным целым числом"
        )
        assert isinstance(intermediate_size, int) and intermediate_size > 0, (
            "intermediate_size должен быть положительным целым числом"
        )
        assert isinstance(dropout, float) and 0.0 <= dropout < 1.0, (
            "dropout должен быть в диапазоне [0.0, 1.0)"
        )

        # --- Сохранение параметров ------------------------------------------------
        # Храним параметры как атрибуты экземпляра
        self.hidden_size = hidden_size
        self.intermediate_size = intermediate_size
        self.dropout_prob = dropout

        # --- Создание слоев ------------------------------------------------------
        # SwiGLU feed-forward сеть
        self.ffn = SwiGLU(
            input_dim=self.hidden_size,
            output_dim=self.hidden_size,
            intermediate_dim=self.intermediate_size
        )

        # Dropout слой для регуляризации
        self.dropout = nn.Dropout(p=self.dropout_prob)


    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:
        """
        Description:
        ---------------
            Применяет преобразование эксперта к входным скрытым состояниям.

            Процесс:
            1. Применение SwiGLU feed-forward сети
            2. Применение dropout для регуляризации

        Args:
        ---------------
            hidden_states: Входной тензор формы (batch_size, seq_len, hidden_size)
                          Скрытые состояния токенов, направленных к этому эксперту

        Returns:
        ---------------
            output: Тензор формы (batch_size, seq_len, hidden_size)
                   Преобразованные скрытые состояния
        """
        # TODO: Примените self.ffn к hidden_states
        # TODO: Примените self.dropout к результату
        # TODO: Верните результат

        # Вопросы для размышления:
        # - Нужен ли residual connection внутри эксперта?
        # - Когда применяется dropout - только при training или всегда?
        # - Как размерности входа и выхода связаны?
        # pass

        # Прямое распространение через SwiGLU FFN
        x = self.ffn(hidden_states)
        # Применение dropout для регуляризации
        x = self.dropout(x)
        
        return x


## 🧰 SimpleMoELayer

In [ ]:
"""
SimpleMoELayer - это учебная версия MoE, которая фокусируется на правильности логики, а не на производительности. Используя простой цикл по токенам, мы избегаем сложных
тензорных операций индексации, делая код понятным и легко отлаживаемым. Это идеальный first step перед оптимизированной версией.
"""


# Стандартная библиотека
from typing import Tuple, Optional

# Сторонние библиотеки
import torch
import torch.nn as nn

# Локальные импорты
from experiments.domain.moe.router import MoERouter
from experiments.domain.moe.expert import Expert


class SimpleMoELayer(nn.Module):
    """
    Description:
    ---------------
        Простая (наивная) реализация MoE Layer для обучения и тестирования.

        Эта версия использует простые циклы вместо оптимизированных тензорных операций.
        Идеально подходит для понимания логики MoE перед переходом к оптимизированной версии.

        Архитектура:
        Input → Router (выбор экспертов) → Dispatch → Experts → Combine → Residual → Output

        Pipeline:
        1. Router: выбирает top_k экспертов для каждого токена
        2. Dispatch: распределяет токены по выбранным экспертам
        3. Process: каждый эксперт обрабатывает свои токены
        4. Combine: собирает результаты с весами от Router
        5. Residual: добавляет входной тензор к выходному

        Для модели 0.6B:
        - num_experts = 8
        - top_k = 2 (каждый токен → 2 эксперта)
        - hidden_size = 512
        - intermediate_size = 2048

    Mathematical Flow:
    ---------------
        x ∈ ℝ^(B×S×H)
            ↓
        Router: (weights, experts_idx, loss) = Router(x)
            weights ∈ ℝ^(B×S×K)      # Веса для K экспертов
            experts_idx ∈ ℤ^(B×S×K)  # Индексы экспертов [0, N)
            ↓
        For each token t in (B×S):
            output[t] = Σ(k=1 to K) weights[t,k] * Expert[experts_idx[t,k]](x[t])
            ↓
        output = output + x  # Residual connection
            ↓
        return output, loss

    Args:
    ---------------
        hidden_size: Размерность входа/выхода (должна совпадать с моделью)
        num_experts: Количество экспертов (8 для модели 0.6B)
        top_k: Количество активных экспертов per token (2 для модели 0.6B)
        intermediate_size: Размерность промежуточного слоя экспертов (обычно 4*hidden_size)
        expert_dropout: Dropout для экспертов (default: 0.0)
        capacity_factor: Фактор емкости для Router (default: 1.25)
        balance_loss_coef: Коэффициент для load balancing loss (default: 0.01)

    Returns (from forward):
    ---------------
        output: Тензор формы (batch_size, seq_len, hidden_size)
                Выходные скрытые состояния после MoE обработки
        balance_loss: Скаляр - load balancing loss для обучения

    Example:
    ---------------
        >>> # Создание MoE Layer для модели 0.6B
        >>> moe = SimpleMoELayer(
        ...     hidden_size=512,
        ...     num_experts=8,
        ...     top_k=2,
        ...     intermediate_size=2048
        ... )
        >>> x = torch.randn(2, 10, 512)  # (batch=2, seq=10, hidden=512)
        >>> output, loss = moe(x, training=True)
        >>> output.shape  # torch.Size([2, 10, 512])
        >>> loss.item()   # Скаляр loss

    Note:
    ---------------
        Это ПРОСТАЯ версия для обучения. Использует циклы вместо
        оптимизированных batch операций. Для production используйте
        оптимизированную версию MoELayer.
    """

    def __init__(
        self,
        hidden_size: int,
        num_experts: int = 8,
        top_k: int = 2,
        intermediate_size: int = 2048,
        expert_dropout: float = 0.0,
        capacity_factor: float = 1.25,
        balance_loss_coef: float = 0.01
    ):
        super().__init__()

        # TODO: Проверьте валидность параметров
        #       - hidden_size > 0
        #       - num_experts > 0
        #       - top_k > 0 и top_k <= num_experts
        #       - intermediate_size > 0
        # TODO: Сохраните параметры как атрибуты класса
        # TODO: Создайте self.router - экземпляр MoERouter
        #       Параметры: hidden_size, num_experts, top_k, capacity_factor, balance_loss_coef
        # TODO: Создайте self.experts - nn.ModuleList из num_experts экспертов
        #       Каждый эксперт: Expert(hidden_size, intermediate_size, expert_dropout)

        # Вопросы для размышления:
        # - Почему используем nn.ModuleList, а не обычный Python list?
        # - Зачем нужен residual connection в MoE Layer?
        # - Как top_k влияет на вычислительную сложность?
        # - Что произойдет, если эксперт получит 0 токенов?
        # pass

        assert hidden_size > 0, "hidden_size должен быть > 0"
        assert num_experts > 0, "num_experts должен быть > 0"
        assert top_k > 0 and top_k <= num_experts, "top_k должен быть > 0 и <= num_experts"
        assert intermediate_size > 0, "intermediate_size должен быть > 0"

        self.hidden_size = hidden_size
        self.num_experts = num_experts
        self.top_k = top_k
        self.intermediate_size = intermediate_size
        self.expert_dropout = expert_dropout
        self.capacity_factor = capacity_factor
        self.balance_loss_coef = balance_loss_coef

        self.router = MoERouter(
            hidden_size=hidden_size,
            num_experts=num_experts,
            top_k=top_k,
            capacity_factor=capacity_factor,
            balance_loss_coef=balance_loss_coef
        )

        self.experts = nn.ModuleList([
            Expert(hidden_size, intermediate_size, expert_dropout)
            for _ in range(num_experts)
        ])

    def forward(
        self,
        hidden_states: torch.Tensor,
        training: bool = True
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Description:
        ---------------
            Применяет MoE трансформацию к входным скрытым состояниям.

            Наивная реализация через циклы (простая, но медленная):
            1. Router выбирает экспертов
            2. Для каждого токена:
               - Берём top_k экспертов
               - Обрабатываем токен каждым экспертом
               - Взвешенное суммирование результатов
            3. Residual connection

        Args:
        ---------------
            hidden_states: Входной тензор формы (batch_size, seq_len, hidden_size)
            training: Флаг режима обучения (для balance loss)

        Returns:
        ---------------
            output: Тензор формы (batch_size, seq_len, hidden_size)
                   Выходные скрытые состояния
            balance_loss: Скаляр - load balancing loss
        """
        # TODO: Шаг 1 - Вызовите self.router
        # TODO: Шаг 2 - Получите размерности hidden_states.shape=
        # TODO: Шаг 3 - Создайте output тензор
        # TODO: Шаг 4 - Dispatch + Process + Combine (наивный подход)
        # TODO: Шаг 5 - output = output + hidden_states
        # TODO: Шаг 6 - Верните (output, balance_loss)

        # Вопросы для размышления:
        # - Почему используем token = hidden_states[b, s:s+1, :] с s:s+1, а не s?
        # - Зачем нужен .item() при извлечении expert_idx и weight?
        # - Что произойдет, если убрать residual connection?
        # - Как можно оптимизировать эти циклы?
        # pass

        # Шаг 1 - Router
        # nn.Module.__call__ обёртка: router(...) автоматически вызывает router.forward(...)
        routing_weights, selected_experts, balance_loss = self.router(hidden_states, training)

        # Шаг 2 - Размерности
        batch_size, seq_len, hidden_size = hidden_states.shape

        # Шаг 3 - Output тензор
        output = torch.zeros(batch_size, seq_len, hidden_size, device=hidden_states.device)

        # Шаг 4 - Dispatch + Process + Combine (наивный подход)
        for b in range(batch_size):
            for s in range(seq_len):
                token = hidden_states[b, s:s+1, :]  # (1, 1, H)
                token_output = torch.zeros(1, 1, hidden_size, device=hidden_states.device)

                for k in range(self.top_k):
                    expert_idx = selected_experts[b, s, k].item()
                    weight = routing_weights[b, s, k].item()

                    expert_output = self.experts[expert_idx](token)  # (1, 1, H)

                    token_output += weight * expert_output  # Взвешенное суммирование

                output[b, s, :] = token_output.squeeze()    # Сохранение результата

        # Шаг 5 - Residual connection
        output = output + hidden_states

        # Шаг 6 - Return
        return output, balance_loss




## ⚙️ OptimizedMoELayer (векторизованный)

In [ ]:
"""
OptimizedMoELayer - векторизованная версия MoE для production использования.

В отличие от SimpleMoELayer (учебная версия с циклами), эта реализация использует
batch operations для максимальной производительности на GPU. Ключевая идея:
вместо обработки токенов по одному, группируем все токены каждого эксперта
и обрабатываем батчем.

Speedup: 2-3x по сравнению с SimpleMoELayer при сохранении численной эквивалентности.
"""


# Стандартная библиотека
from typing import Tuple, Optional

# Сторонние библиотеки
import torch
import torch.nn as nn

# Локальные импорты
from experiments.domain.moe.router import MoERouter
from experiments.domain.moe.expert import Expert


class OptimizedMoELayer(nn.Module):
    """
    Description:
    ---------------
        Оптимизированная (векторизованная) реализация MoE Layer для production.

        Эта версия использует batch operations вместо циклов для максимальной
        производительности на GPU. API полностью совместим с SimpleMoELayer.

        Архитектура (3 фазы):
        Input → Router → Phase 1 (Flatten) → Phase 2 (Parallel Process) →
        Phase 3 (Combine) → Residual → Output

        Pipeline:
        1. Router: выбирает top_k экспертов для каждого токена
        2. Phase 1 (Flatten):
           - Expand токены для K выборов: (B,S,H) → (B,S,K,H)
           - Flatten всё в 1D: (B,S,K,H) → (B*S*K, H)
        3. Phase 2 (Parallel Process):
           - Для каждого эксперта: batch обработка всех его токенов
           - Boolean masking: experts_flat == expert_idx
           - Взвешивание: output * routing_weights
        4. Phase 3 (Combine):
           - Reshape: (B*S*K, H) → (B,S,K,H)
           - Sum по оси K: (B,S,K,H) → (B,S,H)
        5. Residual: добавляет входной тензор к выходному

        Для модели 0.6B:
        - num_experts = 8
        - top_k = 2 (каждый токен → 2 эксперта)
        - hidden_size = 512
        - intermediate_size = 2048

    Mathematical Flow:
    ---------------
        x ∈ ℝ^(B×S×H)
            ↓
        Router: (weights, experts_idx, loss) = Router(x)
            weights ∈ ℝ^(B×S×K)      # Веса для K экспертов
            experts_idx ∈ ℤ^(B×S×K)  # Индексы экспертов [0, N)
            ↓
        Flatten: x_flat ∈ ℝ^(B*S*K × H)
            ↓
        For each expert i in parallel:
            mask_i = (experts_idx == i)
            tokens_i = x_flat[mask_i]
            outputs_i = Expert_i(tokens_i) * weights[mask_i]
            ↓
        Combine: reshape → sum(K) → (B×S×H)
            ↓
        output = output + x  # Residual connection
            ↓
        return output, loss

    Args:
    ---------------
        hidden_size: Размерность входа/выхода (должна совпадать с моделью)
        num_experts: Количество экспертов (8 для модели 0.6B)
        top_k: Количество активных экспертов per token (2 для модели 0.6B)
        intermediate_size: Размерность промежуточного слоя экспертов (обычно 4*hidden_size)
        expert_dropout: Dropout для экспертов (default: 0.0)
        capacity_factor: Фактор емкости для Router (default: 1.25)
        balance_loss_coef: Коэффициент для load balancing loss (default: 0.01)

    Returns (from forward):
    ---------------
        output: Тензор формы (batch_size, seq_len, hidden_size)
                Выходные скрытые состояния после MoE обработки
        balance_loss: Скаляр - load balancing loss для обучения

    Example:
    ---------------
        >>> # Создание оптимизированной MoE Layer для модели 0.6B
        >>> moe = OptimizedMoELayer(
        ...     hidden_size=512,
        ...     num_experts=8,
        ...     top_k=2,
        ...     intermediate_size=2048
        ... )
        >>> x = torch.randn(2, 10, 512)  # (batch=2, seq=10, hidden=512)
        >>> output, loss = moe(x, training=True)
        >>> output.shape  # torch.Size([2, 10, 512])
        >>> loss.item()   # Скаляр loss

    Note:
    ---------------
        Эта версия для PRODUCTION использования. Использует векторизованные
        batch операции для максимальной производительности. Численно эквивалентна
        SimpleMoELayer (до точности float32).
    """

    def __init__(
        self,
        hidden_size: int,
        num_experts: int = 8,
        top_k: int = 2,
        intermediate_size: int = 2048,
        expert_dropout: float = 0.0,
        capacity_factor: float = 1.25,
        balance_loss_coef: float = 0.01
    ):
        super().__init__()

        # TODO(human): Валидация параметров
        # TODO(human): Сохраните параметры как атрибуты класса
        # TODO(human): Создайте self.router - экземпляр MoERouter
        # TODO(human): Создайте self.experts - nn.ModuleList из num_experts экспертов

        # Вопросы для размышления:
        # - Почему используем nn.ModuleList, а не обычный Python list?
        # - Будет ли эта версия API-совместима с SimpleMoELayer?
        # - Какие параметры влияют на memory usage?
        # pass

        assert hidden_size > 0, "hidden_size должен быть > 0"
        assert num_experts > 0, "num_experts должен быть > 0"
        assert top_k > 0 and top_k <= num_experts, "top_k должен быть > 0 и <= num_experts"
        assert intermediate_size > 0, "intermediate_size должен быть > 0"

        self.hidden_size = hidden_size
        self.num_experts = num_experts
        self.top_k = top_k
        self.intermediate_size = intermediate_size
        self.expert_dropout = expert_dropout
        self.capacity_factor = capacity_factor
        self.balance_loss_coef = balance_loss_coef

        self.router = MoERouter(
            hidden_size = hidden_size,
            num_experts = num_experts,
            top_k = top_k,
            capacity_factor = capacity_factor,
            balance_loss_coef = balance_loss_coef
        )

        self.experts = nn.ModuleList([
            Expert(
                hidden_size = hidden_size,
                intermediate_size = intermediate_size,
                dropout = expert_dropout
            )
            for _ in range(num_experts)
        ])

    def forward(
        self,
        hidden_states: torch.Tensor,
        training: bool = True
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Description:
        ---------------
            Применяет векторизованную MoE трансформацию к входным скрытым состояниям.

            Оптимизированная реализация через batch operations:
            1. Router выбирает экспертов для всех токенов
            2. Phase 1 - Flatten: (B,S,H) → (B,S,K,H) → (B*S*K, H)
            3. Phase 2 - Parallel Process: batch обработка каждым экспертом
            4. Phase 3 - Combine: (B*S*K, H) → (B,S,K,H) → sum(K) → (B,S,H)
            5. Residual connection

        Args:
        ---------------
            hidden_states: Входной тензор формы (batch_size, seq_len, hidden_size)
            training: Флаг режима обучения (для balance loss)

        Returns:
        ---------------
            output: Тензор формы (batch_size, seq_len, hidden_size)
                   Выходные скрытые состояния
            balance_loss: Скаляр - load balancing loss

        Shape Transformations:
        ---------------
            hidden_states:     (B, S, H)
                ↓ unsqueeze(2)
            tokens:            (B, S, 1, H)
                ↓ expand(-1, -1, K, -1)
            tokens_expanded:   (B, S, K, H)
                ↓ reshape(-1, H)
            tokens_flat:       (B*S*K, H)
                ↓ expert processing + weighting
            expert_outputs:    (B*S*K, H)
                ↓ reshape(B, S, K, H)
            expert_outputs:    (B, S, K, H)
                ↓ sum(dim=2)
            combined:          (B, S, H)
                ↓ residual
            output:            (B, S, H)
        """
        # TODO(human): Шаг 0 - Router
        #       Получите routing_weights, selected_experts, balance_loss от self.router

        # TODO(human): Шаг 1 - Flatten для batch processing
        #       1.1. Извлеките размерности: hidden_states.shape
        #       1.2. Сохраните self.top_k
        #       1.3. Expand токены для K выборов:
        #            Преобразуйте hidden_states: (B, S, H) → (B, S, 1, H) → (B, S, K, H)
        #       1.4. Flatten всё в 1D:
        #            tokens_flat  = (B*S*K, H)
        #            weights_flat = (B*S*K,)
        #            experts_flat = (B*S*K,)

        # TODO(human): Шаг 2 - Parallel Expert Processing
        #       2.1. Создайте output тензор: expert_outputs
        #       2.2. Для каждого эксперта:
        #            a) Создайте boolean маску
        #            b) Проверьте: (skip пустых экспертов)
        #            c) Извлеките токены эксперта
        #            d) Обработайте батчем
        #            e) Взвесьте по routing_weights
        #            f) Запишите обратно в weighted_output
        
        # TODO(human): Шаг 3 - Combine - суммируем K вкладов для каждого токена
        # TODO(human): Шаг 4 - Residual connection
        # TODO(human): Шаг 5 - Return
        #       return output, balance_loss

        # Вопросы для размышления:
        # - Почему мы используем unsqueeze(2).expand(), а не repeat()?
        # - Зачем проверять mask.sum() > 0 перед вызовом эксперта?
        # - Как weights_flat[mask].unsqueeze(-1) влияет на broadcasting?
        # - Почему sum(dim=2) корректно объединяет K вкладов?
        # - В чём разница между этой реализацией и SimpleMoELayer в плане памяти?
        # pass

        # ════════════════════════════════════════════════════════════════════
        # Шаг 0: Router - выбираем top_k экспертов для каждого токена
        # ════════════════════════════════════════════════════════════════════
        routing_weights, selected_experts, load_balance_loss = self.router(hidden_states, training)

        # ════════════════════════════════════════════════════════════════════
        # Шаг 1: Flatten - подготовка для batch processing
        # ════════════════════════════════════════════════════════════════════
        # Извлекаем размерности входного тензора
        batch_size, seq_len, hidden_size = hidden_states.shape

        # Сохраняем top_k для удобства
        top_k = self.top_k

        # ────────────────────────────────────────────────────────────────────
        # Шаг 1.1: Expand токены для K выборов (memory-efficient дублирование)
        # ────────────────────────────────────────────────────────────────────
        # Трансформация: (B, S, H) → (B, S, 1, H) → (B, S, K, H)
        # unsqueeze(2): добавляем новую ось размера 1 на позиции 2
        # expand(): "растягиваем" ось 1 → K (БЕЗ копирования в памяти!)
        # Результат: каждый токен виртуально дублирован K раз (для K экспертов)
        # Исходный токен (вектор):
        # hidden_states[b=0, s=0] = [1, 2, 3, 4]  # shape: (H=4,)

        # # После unsqueeze(2).expand(..., K=2, ...):
        # tokens[b=0, s=0] = [
        #     [1, 2, 3, 4],  # k=0 (копия для первого эксперта)
        #     [1, 2, 3, 4]   # k=1 (копия для второго эксперта)
        # ]  # shape: (K=2, H=4) - это матрица!
        tokens = hidden_states.unsqueeze(2).expand(batch_size, seq_len, top_k, hidden_size)

        # ────────────────────────────────────────────────────────────────────
        # Шаг 1.2: Flatten всё в 1D для векторизованной обработки
        # ────────────────────────────────────────────────────────────────────
        # Flatten: (B, S, K, H) → (B*S*K, H)
        # "Разворачиваем" 4D тензор в 2D матрицу (список векторов)

        # До reshape (4D): tokens.shape = (B=2, S=3, K=2, H=4)
        # tokens = [
        #   # Batch 0:
        #   [ [[1,2,3,4], [1,2,3,4]],    # s=0: матрица 2×4
        #     [[5,6,7,8], [5,6,7,8]],    # s=1: матрица 2×4
        #     [[9,10,11,12], [9,10,11,12]] ], # s=2: матрица 2×4
        #   # Batch 1: аналогично...
        # ]

        # После reshape (2D): tokens_flat.shape = (B*S*K=12, H=4)
        # tokens_flat = [
        #   [1, 2, 3, 4],      # индекс 0: (b=0, s=0, k=0)
        #   [1, 2, 3, 4],      # индекс 1: (b=0, s=0, k=1)
        #   [5, 6, 7, 8],      # индекс 2: (b=0, s=1, k=0)
        #   [5, 6, 7, 8],      # индекс 3: (b=0, s=1, k=1)
        #   [9, 10, 11, 12],   # индекс 4: (b=0, s=2, k=0)
        #   [9, 10, 11, 12],   # индекс 5: (b=0, s=2, k=1)
        #   # ... batch 1: индексы 6-11
        # ]
        # ⚠️ Порядок flatten: сначала batch, потом sequence, потом K
        tokens_flat = tokens.reshape(-1, hidden_size)  # (B*S*K, H)

        # Flatten весов: (B, S, K) → (B*S*K,)
        # weights_flat[i] = вес для tokens_flat[i]
        weights_flat = routing_weights.reshape(-1)     # (B*S*K,)

        # Flatten индексов экспертов: (B, S, K) → (B*S*K,)
        # experts_flat[i] = какой эксперт должен обработать tokens_flat[i]
        experts_flat = selected_experts.reshape(-1)    # (B*S*K,)

        # ⚠️ ВАЖНО: После flatten все 3 тензора синхронизированы по индексу:
        #   tokens_flat[i]   - токен для обработки
        #   weights_flat[i]  - вес результата при комбинировании
        #   experts_flat[i]  - индекс эксперта [0, num_experts)

        # ════════════════════════════════════════════════════════════════════
        # Шаг 2: Parallel Expert Processing - batch обработка каждым экспертом
        # ════════════════════════════════════════════════════════════════════
        # Инициализируем выходной тензор нулями (будем заполнять по маскам)
        # Каждый токен в tokens_flat будет обработан ровно 1 раз (по своему эксперту)
        expert_outputs = torch.zeros_like(tokens_flat)  # (B*S*K, H)

        # Цикл по экспертам: каждый обрабатывает свою группу токенов батчем
        # ⚠️ ВАЖНО: Это единственный цикл в оптимизированной версии!
        #   SimpleMoELayer: 3 вложенных цикла (batch × sequence × top_k)
        #   OptimizedMoELayer: 1 цикл (num_experts), внутри - batch operations
        for expert_idx in range(self.num_experts):
            # ────────────────────────────────────────────────────────────────
            # Шаг 2.1: Boolean masking - находим все токены для этого эксперта
            # ────────────────────────────────────────────────────────────────
            # mask = True для позиций, где experts_flat == expert_idx
            # Например, если expert_idx=3, то mask выделит все токены,
            # которые Router назначил третьему эксперту
            mask = (experts_flat == expert_idx)  # (B*S*K,) - boolean тензор

            # Пример mask для expert_idx=0:
            # experts_flat = [0, 2, 0, 5, 0, 1, ...]
            # mask         = [T, F, T, F, T, F, ...]
            # Где T означает "этот токен для эксперта 0"

            # ────────────────────────────────────────────────────────────────
            # Шаг 2.2: Skip пустых экспертов (оптимизация)
            # ────────────────────────────────────────────────────────────────
            # Если Router не назначил ни одного токена этому эксперту, пропускаем
            # Это экономит время на forward pass пустого эксперта
            if mask.sum() > 0:
                # ────────────────────────────────────────────────────────────
                # Шаг 2.3: Извлечение токенов эксперта через boolean indexing
                # ────────────────────────────────────────────────────────────
                # Извлекаем только те токены, для которых mask == True
                # Это создаёт новый тензор (компактный, без пустых мест)
                expert_tokens = tokens_flat[mask]  # (num_selected_tokens, H)

                # Пример:
                # tokens_flat = [
                #   [1, 2, 3, 4],   # индекс 0 (mask=True для expert_idx=0)
                #   [5, 6, 7, 8],   # индекс 1 (mask=False)
                #   [9, 10, 11, 12] # индекс 2 (mask=True для expert_idx=0)
                # ]
                # expert_tokens = [
                #   [1, 2, 3, 4],   # из индекса 0
                #   [9, 10, 11, 12] # из индекса 2
                # ]  # shape: (2, H) - только выбранные токены!

                # ────────────────────────────────────────────────────────────
                # Шаг 2.4: Batch обработка экспертом
                # ────────────────────────────────────────────────────────────
                # Вызываем эксперта ОДИН РАЗ для ВСЕХ его токенов
                # Это ключ к ускорению: вместо N вызовов - 1 вызов с батчем
                output = self.experts[expert_idx](expert_tokens)  # (num_selected_tokens, H)

                # ────────────────────────────────────────────────────────────
                # Шаг 2.5: Взвешивание по routing weights
                # ────────────────────────────────────────────────────────────
                # Извлекаем веса для выбранных токенов (используя ту же маску)
                # unsqueeze(-1): (num_tokens,) → (num_tokens, 1) для broadcasting
                expert_weights = weights_flat[mask].unsqueeze(-1)  # (num_selected_tokens, 1)

                # Broadcasting: (num_tokens, H) * (num_tokens, 1) → (num_tokens, H)
                # Каждая строка output умножается на свой скалярный вес
                # Пример:
                # output = [[1, 2], [3, 4]]       # (2, 2)
                # weights = [[0.7], [0.3]]        # (2, 1)
                # result = [[0.7, 1.4], [0.9, 1.2]] # (2, 2)
                weighted_output = output * expert_weights  # (num_selected_tokens, H)

                # ────────────────────────────────────────────────────────────
                # Шаг 2.6: Запись обратно в исходные позиции
                # ────────────────────────────────────────────────────────────
                # Используем ту же маску для записи результатов обратно
                # Boolean indexing работает и для присваивания!
                expert_outputs[mask] = weighted_output

                # Визуализация процесса заполнения:
                # До обработки: expert_outputs = [[0,0,0,0], [0,0,0,0], [0,0,0,0]]
                # После expert_idx=0: expert_outputs = [[1,2,3,4], [0,0,0,0], [9,10,11,12]]
                # После expert_idx=1: expert_outputs = [[1,2,3,4], [5,6,7,8], [9,10,11,12]]

        # ════════════════════════════════════════════════════════════════════
        # Шаг 3: Combine - суммируем K вкладов для каждого токена
        # ════════════════════════════════════════════════════════════════════
        # ────────────────────────────────────────────────────────────────────
        # Шаг 3.1: Reshape обратно в 4D структуру
        # ────────────────────────────────────────────────────────────────────
        # Восстанавливаем исходную структуру: (B*S*K, H) → (B, S, K, H)
        # Это обратная операция к flatten из Шага 1.2
        expert_outputs = expert_outputs.reshape(batch_size, seq_len, top_k, hidden_size)

        # Визуализация reshape:
        # До reshape (2D): expert_outputs.shape = (B*S*K=12, H=4)
        # expert_outputs_flat = [
        #   [1, 2, 3, 4],      # (b=0, s=0, k=0) - вклад эксперта 0
        #   [0.5, 1, 1.5, 2],  # (b=0, s=0, k=1) - вклад эксперта 2
        #   [5, 6, 7, 8],      # (b=0, s=1, k=0)
        #   [2.5, 3, 3.5, 4],  # (b=0, s=1, k=1)
        #   ...
        # ]
        #
        # После reshape (4D): expert_outputs.shape = (B=2, S=3, K=2, H=4)
        # expert_outputs = [
        #   [ # Batch 0
        #     [[1,2,3,4], [0.5,1,1.5,2]],           # s=0: K=2 вклада
        #     [[5,6,7,8], [2.5,3,3.5,4]],           # s=1: K=2 вклада
        #     [[9,10,11,12], [4.5,5,5.5,6]]         # s=2: K=2 вклада
        #   ],
        #   [ # Batch 1: аналогично... ]
        # ]

        # ────────────────────────────────────────────────────────────────────
        # Шаг 3.2: Суммирование по оси K (объединение вкладов экспертов)
        # ────────────────────────────────────────────────────────────────────
        # Суммируем по оси K (dim=2): каждый токен получает сумму от K экспертов
        # (B, S, K, H) → sum(dim=2) → (B, S, H)
        combined = expert_outputs.sum(dim=2)  # (B, S, H)

        # Визуализация суммирования:
        # До sum: expert_outputs[b=0, s=0] = [[1,2,3,4], [0.5,1,1.5,2]]  # (K=2, H=4)
        # После sum: combined[b=0, s=0] = [1.5, 3, 4.5, 6]  # (H=4) - сумма вкладов!
        #
        # Это и есть "взвешенное комбинирование" выходов экспертов.
        # Каждый эксперт внёс свой вклад (умноженный на routing_weight),
        # а мы их суммируем для финального представления токена.

        # ════════════════════════════════════════════════════════════════════
        # Шаг 4: Residual Connection
        # ════════════════════════════════════════════════════════════════════
        # Добавляем исходный вход к выходу MoE слоя
        # Это стандартная практика в Transformer архитектурах для:
        # 1. Стабилизации обучения (градиенты текут напрямую)
        # 2. Сохранения исходной информации (слой может научиться "не делать ничего")
        # 3. Улучшения сходимости (каждый слой учит только дельту)
        output = combined + hidden_states  # (B, S, H) + (B, S, H) → (B, S, H)

        # ════════════════════════════════════════════════════════════════════
        # Шаг 5: Return - возвращаем результат и loss
        # ════════════════════════════════════════════════════════════════════
        # output: финальные скрытые состояния после MoE трансформации
        # load_balance_loss: метрика для обучения (стимулирует равномерное использование экспертов)
        return output, load_balance_loss



## 🧱 MoE Transformer Block

In [ ]:
# Стандартная библиотека
from typing import Optional, Tuple, Union

# Сторонние библиотеки
import torch
import torch.nn as nn

# Локальные импорты
import sys
import os
parent_dir = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
sys.path.insert(0, parent_dir)
from normalization.rmsnorm import RMSNorm
from attention.gqa import GroupedQueryAttention
from moe.moe_layer import SimpleMoELayer


class MoETransformerBlock(nn.Module):
    """
    Description:
    ---------------
        MoE Transformer Block для архитектуры Qwen3. Использует Pre-Norm архитектуру
        с Grouped-Query Attention и SimpleMoELayer вместо обычного FFN.

        Архитектура блока:
        Input → RMSNorm → GQA → Residual → RMSNorm → SimpleMoELayer → Residual → Output
                                                           ↓
                                                     balance_loss

        Отличие от обычного TransformerBlock:
        - SwiGLU FFN заменён на SimpleMoELayer
        - Forward возвращает (output, balance_loss) вместо просто output
        - balance_loss используется для обучения (предотвращение коллапса экспертов)

    Args:
    ---------------
        hidden_size: Размерность скрытого состояния (d_model)
        num_query_groups: Количество групп запросов для GQA
        num_attention_heads: Количество голов внимания для key/value
        num_experts: Количество экспертов в MoE Layer (default: 8)
        top_k: Количество активных экспертов per token (default: 2)
        intermediate_size: Размерность промежуточного слоя в экспертах (default: 4 * hidden_size)
        expert_dropout: Dropout для экспертов (default: 0.0)
        balance_loss_coef: Коэффициент для load balancing loss (default: 0.01)

    Returns (from forward):
    ---------------
        Если use_cache или output_attentions:
            (hidden_states, balance_loss, present_key_value, attn_weights)
        Иначе:
            (hidden_states, balance_loss)

    Example:
    ---------------
        >>> # Создание MoE Transformer Block
        >>> block = MoETransformerBlock(
        ...     hidden_size=512,
        ...     num_query_groups=8,
        ...     num_attention_heads=16,
        ...     num_experts=8,
        ...     top_k=2
        ... )
        >>> x = torch.randn(2, 10, 512)  # (batch, seq, hidden)
        >>> output, balance_loss = block(x)
        >>> output.shape  # torch.Size([2, 10, 512])
        >>> balance_loss.item()  # Скаляр loss

    Note:
    ---------------
        balance_loss должен быть добавлен к общему loss модели во время обучения:
        total_loss = language_model_loss + balance_loss
    """

    def __init__(
        self,
        hidden_size: int,
        num_query_groups: int,
        num_attention_heads: int,
        num_experts: int = 8,
        top_k: int = 2,
        intermediate_size: Optional[int] = None,
        expert_dropout: float = 0.0,
        balance_loss_coef: float = 0.01
    ):
        super().__init__()

        # TODO: Проверьте валидность параметров
        #       Подсказка: посмотрите TransformerBlock (строки 53-80)
        #       Добавьте проверку для MoE: top_k <= num_experts

        # TODO: Вычислите intermediate_size если не указан
        #       Подсказка: какое стандартное соотношение к hidden_size?

        # TODO: Сохраните параметры как атрибуты класса
        #       Подсказка: self.hidden_size = ..., self.num_experts = ..., и т.д.

        # TODO: Создайте 4 компонента (см. TransformerBlock строки 91-102):
        #       - self.attention_norm (какой тип нормализации?)
        #       - self.attention (какой механизм внимания?)
        #       - self.ffn_norm (снова нормализация)
        #       - self.moe_layer (вместо self.feed_forward!)
        #
        #       Вопрос: какие параметры нужны SimpleMoELayer?

        # Вопросы для размышления:
        # - Почему мы заменяем FFN на MoE Layer?
        # - Как balance_loss влияет на обучение модели?
        # - Что произойдёт, если не добавлять balance_loss к общему loss?
        # pass

        # --- Валидация параметров -------------------------------------------------
        assert (
            isinstance(hidden_size, int) and hidden_size > 0
        ), "hidden_size должен быть положительным целым числом"
        assert (
            isinstance(num_query_groups, int) and num_query_groups > 0
        ), "num_query_groups должен быть положительным целым числом"
        assert (
            isinstance(num_attention_heads, int) and num_attention_heads > 0
        ), "num_attention_heads должен быть положительным целым числом"

        # MoE специфичные проверки
        assert (
            isinstance(num_experts, int) and num_experts > 0
        ), "num_experts должен быть положительным целым числом"
        assert (
            top_k > 0 and top_k <= num_experts
        ), "top_k должен быть > 0 и <= num_experts"

        # Ключевая проверка для GQA архитектуры:
        assert (
            num_attention_heads % num_query_groups == 0
        ), (
            "num_attention_heads должен делиться на num_query_groups для "
            "корректной работы GQA"
        )
        # Проверка делимости hidden_size на число голов
        # Тензор скрытого состояния должен равномерно делиться на число голов
        assert (
            hidden_size % num_attention_heads == 0
        ), "hidden_size должен делиться на num_attention_heads"

        # Проверка intermediate_size если указан
        if intermediate_size is not None:
            assert (
                isinstance(intermediate_size, int) and intermediate_size > 0
            ), "intermediate_size должен быть положительным целым числом"

        # --- Инициализация атрибутов ----------------------------------------------
        self.hidden_size         = hidden_size
        self.num_query_groups    = num_query_groups
        self.num_attention_heads = num_attention_heads
        self.num_experts         = num_experts
        self.top_k               = top_k
        self.intermediate_size = (
            intermediate_size if intermediate_size is not None else 4 * hidden_size
        )
        self.expert_dropout    = expert_dropout
        self.balance_loss_coef = balance_loss_coef

        # --- Компоненты нормализации и подблоков ----------------------------------
        self.attention_norm = RMSNorm(hidden_size)
        self.attention = GroupedQueryAttention(
            hidden_size=hidden_size,
            num_query_groups=num_query_groups,
            num_attention_heads=num_attention_heads,
        )
        self.ffn_norm  = RMSNorm(hidden_size)
        self.moe_layer = SimpleMoELayer(
            hidden_size = hidden_size,
            num_experts = num_experts,
            top_k = top_k,
            intermediate_size = self.intermediate_size,
            expert_dropout = expert_dropout,
            balance_loss_coef = balance_loss_coef
        )


    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        past_key_value: Optional[Tuple[torch.Tensor, torch.Tensor]] = None,
        output_attentions: bool = False,
        use_cache: bool = False,
        training: bool = True
    ) -> Union[Tuple[torch.Tensor, torch.Tensor], Tuple[torch.Tensor, torch.Tensor, Optional[Tuple], Optional[torch.Tensor]]]:
        """
        Description:
        ---------------
            Применяет MoE Transformer блок к входному тензору.

            Pipeline:
            1. RMSNorm → GQA → Residual
            2. RMSNorm → SimpleMoELayer → Residual
            3. Return (output, balance_loss)

        Args:
        ---------------
            hidden_states: Входной тензор формы (batch_size, seq_len, hidden_size)
            attention_mask: Маска внимания (optional)
            position_ids: Позиционные индексы для RoPE (optional)
            past_key_value: Кэш key/value для генерации (optional)
            output_attentions: Возвращать ли attention weights (default: False)
            use_cache: Использовать ли KV cache (default: False)
            training: Режим обучения для balance_loss (default: True)

        Returns:
        ---------------
            Если use_cache или output_attentions:
                (hidden_states, balance_loss, present_key_value, attn_weights)
            Иначе:
                (hidden_states, balance_loss)
        """
        # TODO: ПЕРВЫЙ RESIDUAL BLOCK - Self-Attention (GQA)
        #       Подсказка: скопируйте из TransformerBlock (строки 143-171)
        #       Структура: residual → norm → attention → residual_add
        #       Внимание: attention может вернуть tuple!

        # TODO: ВТОРОЙ RESIDUAL BLOCK - MoE Feed-Forward
        #       Подсказка: структура как в TransformerBlock (строки 177-187)
        #       НО: self.feed_forward → self.moe_layer
        #       ВАЖНО: moe_layer возвращает (output, balance_loss) - tuple!
        #       Не забудьте передать параметр training

        # TODO: RETURN
        #       Вопрос: сколько значений нужно вернуть?
        #       - Всегда: (hidden_states, balance_loss)
        #       - Если use_cache/output_attentions: добавьте present_kv и attn_weights
        #       Подсказка: посмотрите TransformerBlock строки 189-192

        # Вопросы для размышления:
        # - Почему balance_loss возвращается вместе с hidden_states?
        # - Как будет собираться balance_loss от всех слоёв модели?
        # - Чем отличается forward от обычного TransformerBlock?
        # pass

        # ──────────────────────────────────────────────────────────────────────────
        # ПЕРВЫЙ RESIDUAL BLOCK: Self-Attention (GQA)
        # ──────────────────────────────────────────────────────────────────────────

        # Сохраняем вход для остаточной связи, что бы потом прибавить к выходу блока
        # p.s. Помогает сохранить информацию о входном тензоре (векторах), чтобы не терять её.
        residual_1 = hidden_states

        # Нормализуем входной тензор
        normed = self.attention_norm(hidden_states)

        # Вызываем модуль группового внимания. Он может вернуть:
        # - только выход (Tensor), либо
        # - кортеж (att_output, present_key_value, attn_weights).
        att_output = self.attention(
            hidden_states=normed,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_value=past_key_value,
            output_attentions=output_attentions,
            use_cache=use_cache,
        )

        if isinstance(att_output, tuple):
            att_output, present_key_value, attn_weights = att_output
        else:
            present_key_value = None
            attn_weights = None

        # Первая residual-связь: складываем вход и выход подблока.
        hidden_states = att_output + residual_1

        # ──────────────────────────────────────────────────────────────────────────
        # ВТОРОЙ RESIDUAL BLOCK: MoE Feed-Forward
        # ──────────────────────────────────────────────────────────────────────────

        residual_2 = hidden_states

        # Нормализуем перед MoE по тем же причинам, что и перед вниманием.
        normed = self.ffn_norm(hidden_states)
        # Применяем MoE слой вместо обычного FFN.
        ffn_output, balance_loss = self.moe_layer(hidden_states=normed, training=training)
        # Вторая residual-связь.
        hidden_states = ffn_output + residual_2

        if use_cache or output_attentions:
            return hidden_states, balance_loss, present_key_value, attn_weights

        return hidden_states, balance_loss


## 🧩 Полная модель Qwen3MoEModel

In [ ]:
"""
Qwen3 MoE Language Model

Полная реализация генеративной языковой модели с MoE архитектурой.
"""
import torch
import torch.nn as nn
from typing import Optional, Tuple
from transformers import GPT2Tokenizer

from .config import Qwen3Config
from ..normalization.rmsnorm import RMSNorm
from ..transformer.moe_transformer_block import MoETransformerBlock


class Qwen3MoEModel(nn.Module):
    """
    Description:
    ---------------
        Полная генеративная языковая модель Qwen3 с MoE архитектурой.

    Архитектура:
    ------------
    Input (batch, seq_len) — token IDs
        ↓
    Token Embedding (batch, seq_len, hidden_size)
        ↓
    N × MoE Transformer Blocks
        ├─ RMSNorm
        ├─ Grouped-Query Attention + RoPE
        ├─ RMSNorm
        └─ SimpleMoELayer (8 экспертов, 2 активных)
        ↓
    Final RMSNorm (batch, seq_len, hidden_size)
        ↓
    LM Head: Linear(hidden_size → vocab_size)
        ↓
    Output Logits (batch, seq_len, vocab_size)

    Args:
    -----
        config: Qwen3Config с параметрами модели

    Attributes:
    -----------
        embed_tokens: Token embedding layer (vocab_size → hidden_size)
        layers: nn.ModuleList из N MoE Transformer блоков
        norm: Final RMSNorm перед LM head
        lm_head: Language modeling head (hidden_size → vocab_size)
        tokenizer: GPT2Tokenizer для encode/decode текста

    Examples:
    ---------
        >>> config = Qwen3Config()
        >>> model = Qwen3MoEModel(config)
        >>>
        >>> # Forward pass
        >>> input_ids = torch.randint(0, config.vocab_size, (2, 10))
        >>> logits, balance_loss = model(input_ids)
        >>> print(logits.shape)  # (2, 10, 50257)
        >>>
        >>> # Generation
        >>> generated = model.generate(input_ids, max_length=50)
        >>> print(generated.shape)  # (2, 50)
    """

    def __init__(self, config: Qwen3Config, tokenizer: Optional[GPT2Tokenizer] = None):
        super().__init__()
        # TODO: Инициализируйте все компоненты модели
        # Вопрос: Какие основные блоки нужны для полной модели?
        # Подсказка: Используйте config для параметров
        
        # TODO: Инициализация tokenizer (если None, загрузите GPT-2)

        # TODO: Преобразование token IDs → continuous vectors
        # Вопрос: Какой PyTorch слой создаёт lookup table размера (vocab_size, hidden_size)?
        # Подсказка: В SimpleMoELayer вы использовали nn.ModuleList. А для embeddings?

        # TODO: Стек из N transformer блоков
        # Вопрос: Как создать список из config.num_layers одинаковых блоков MoETransformerBlock(config)?
        # Подсказка: Вспомните, как в SimpleMoELayer создавались эксперты

        # TODO: Финальная нормализация скрытых состояний
        # Вопрос: Какой компонент нормализации вы реализовали на первых этапах?
        # Подсказка: Принимает один аргумент — размерность для нормализации

        # TODO: Проекция hidden_size → vocab_size для предсказания токенов
        # Вопрос: Какой слой проецирует векторы из одного пространства в другое?
        # Подсказка: В LM обычно используют bias=False в финальной проекции

        # Вопросы для размышления:
        # - Почему все четыре компонента должны быть атрибутами класса (self.*)?
        # - Что произойдёт, если использовать Python list вместо nn.ModuleList?
        # - Почему размерность embedding должна совпадать с hidden_size блоков?

        self.config = config

        # Инициализация tokenizer для text ↔ token_ids
        # Используем GPT-2 tokenizer (vocab_size=50257), совместимый с config
        if tokenizer is None:
            self.tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
            # Важно: добавляем pad_token, т.к. GPT-2 изначально его не имеет
            self.tokenizer.pad_token = self.tokenizer.eos_token
        else:
            self.tokenizer = tokenizer

        # Token Embedding Layer: преобразование token IDs → continuous vectors
        # Создаёт таблицу размера (y = vocab_size, x = hidden_size)
        self.embed_tokens = nn.Embedding(
            num_embeddings = config.vocab_size, 
            embedding_dim  = config.hidden_size
            )

        # Стек из N transformer блоков
        # Каждый блок содержит: RMSNorm → GQA → RMSNorm → SimpleMoELayer
        self.layers = nn.ModuleList([
            MoETransformerBlock(
                hidden_size=config.hidden_size,
                num_query_groups=config.num_key_value_heads,
                num_attention_heads=config.num_attention_heads,
                num_experts=config.num_experts,
                top_k=config.top_k,
                intermediate_size=config.intermediate_size,
                expert_dropout = config.dropout,
                balance_loss_coef=config.balance_loss_coef
            ) for _ in range(config.num_layers)
        ])

        # Финальная нормализация скрытых состояний
        self.norm = RMSNorm(normalized_shape = self.config.hidden_size)

        # Проекция hidden_size → vocab_size для предсказания токенов
        # y = x Aᵀ (без bias для LM head)
        self.lm_head = nn.Linear(
            in_features  = self.config.hidden_size,
            out_features = self.config.vocab_size,
            bias         = False
        )

        # Инициализация весов
        self._init_weights()

    def _init_weights(self):
        """
        Description:
        ---------------
            Инициализация весов модели.

        Стратегия:
        ----------
        - Embeddings: normal distribution N(0, initializer_range)
        - Linear layers: уже инициализированы в sub-модулях
        - LM Head: normal distribution N(0, initializer_range)
        """
        # TODO: Инициализируйте веса эмбеддингов и LM head

        # Вопросы для размышления:
        # - Почему инициализация важна для стабильного обучения?
        # - Как влияет stddev на обучение?
        # - Почему линейные слои не требуют дополнительной инициализации?

        # Embedding initialization
        nn.init.normal_(self.embed_tokens.weight, mean=0.0, std=self.config.initializer_range)

        # LM Head initialization
        nn.init.normal_(self.lm_head.weight, mean=0.0, std=self.config.initializer_range)

    def forward(
        self,
        input_ids: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Description:
        ---------------
            Forward pass модели.

        Pipeline:
        ---------
        1. Token IDs → Embeddings (lookup)
        2. Embeddings → Transformer Blocks (N раз)
        3. Hidden States → Final Norm
        4. Normalized States → LM Head → Logits
        5. Accumulate balance loss из всех MoE блоков

        Args:
        -----
            input_ids: Тензор token IDs формы (batch_size, seq_len)
            attention_mask: Опциональная маска внимания (batch_size, seq_len)
                           1 = attend, 0 = ignore. По умолчанию None (все токены видимы)

        Returns:
        --------
            logits: Тензор логитов формы (batch_size, seq_len, vocab_size)
                   Вероятности следующего токена для каждой позиции
            balance_loss: Скалярный тензор, сумма balance losses из всех MoE слоёв
                         Используется для load balancing экспертов

        Shape Flow:
        -----------
            input_ids: (B, S) → embeddings: (B, S, H)
            → transformer blocks → hidden_states: (B, S, H)
            → norm → normalized: (B, S, H)
            → lm_head → logits: (B, S, V)

        Examples:
        ---------
            >>> model = Qwen3MoEModel(config)
            >>> input_ids = torch.randint(0, 50257, (4, 32))  # batch=4, seq=32
            >>> logits, loss = model(input_ids)
            >>> print(f"Logits: {logits.shape}, Loss: {loss.item():.4f}")
            Logits: torch.Size([4, 32, 50257]), Loss: 0.0234
        """
        # TODO: Преобразуйте input_ids → embeddings через эмбендинг слой
        # TODO: Инициализируйте total_balance_loss нулевым тензором на device embeddings
        # TODO: Пройдите циклом по self.layers, накапливая balance_loss
        # TODO: Примените финальную нормализацию self.norm
        # TODO: Спроецируйте через self.lm_head в vocab space
        # TODO: Верните (logits, total_balance_loss)

        # Вопросы для размышления:
        # - Почему важно указать device при создании total_balance_loss?
        # - Что возвращает каждый MoE блок?
        # - Чем logits отличаются от вероятностей?
        # pass

        # Преобразование token IDs в embeddings через lookup table
        embeddings = self.embed_tokens(input_ids)
        
        # Инициализация тензора для накопления balance loss из всех MoE блоков
        # Важно: используем device от embeddings для совместимости с GPU/CPU
        total_balance_loss = torch.tensor(
            data=0.0,
            device=embeddings.device
        )
        
        # Проход через все transformer блоки с накоплением balance loss
        # Каждый layer - это экземпляр MoETransformerBlock, который возвращает:
        # 1. Обработанные embeddings (RMSNorm → GQA → RMSNorm → SimpleMoELayer)
        # 2. Balance loss для load balancing экспертов
        for layer in self.layers:
            embeddings, balance_loss = layer(embeddings, attention_mask)
            total_balance_loss += balance_loss

        # Финальная нормализация скрытых состояний перед LM head
        final_norm = self.norm(embeddings)
        
        # Проекция в пространство словаря для предсказания следующего токена
        logits = self.lm_head(final_norm)

        return logits, total_balance_loss

    def generate(
        self,
        input_ids: torch.Tensor,
        max_length: int = 100,
        temperature: float = 1.0,
        top_k: Optional[int] = None,
        top_p: Optional[float] = None,
        do_sample: bool = True,
    ) -> torch.Tensor:
        """
        Description:
        ---------------
            Автогрессивная генерация текста.

        Стратегия:
        ----------
        1. Начинаем с input_ids (prompt)
        2. В цикле (до max_length):
           a. Forward pass: получаем logits для следующего токена
           b. Применяем temperature/top-k/top-p
           c. Сэмплируем следующий токен
           d. Добавляем токен к последовательности
        3. Возвращаем сгенерированную последовательность

        Args:
        -----
            input_ids: Начальная последовательность (prompt) формы (batch, seq_len)
            max_length: Максимальная длина генерируемой последовательности
            temperature: Температура для сэмплирования (>1 = более случайно, <1 = более детерминированно)
            top_k: Оставить только k самых вероятных токенов (nucleus sampling)
            top_p: Оставить минимальное множество токенов с суммарной вероятностью ≥ p
            do_sample: True = сэмплирование, False = greedy (argmax)

        Returns:
        --------
            generated_ids: Сгенерированная последовательность формы (batch, max_length)

        Examples:
        ---------
            >>> # Greedy decoding
            >>> output = model.generate(input_ids, max_length=50, do_sample=False)
            >>>
            >>> # Nucleus sampling (top-p)
            >>> output = model.generate(input_ids, temperature=0.8, top_p=0.9)
            >>>
            >>> # Top-k sampling
            >>> output = model.generate(input_ids, temperature=1.0, top_k=50)
        """
        # TODO: Инициализация переменных для генерации
        # - Скопировать input_ids для безопасного изменения
        # - Инициализировать key-value cache (если используется)
        # - Вычислить начальную длину последовательности
        # - Подготовить attention mask для начальной последовательности
        
        # TODO: Основной цикл автогрессивной генерации
        # - while current_length < max_length:
        #   a. Forward pass: получить logits для последнего токена
        #   b. Извлечь logits только для последней позиции (shape: [batch, vocab_size])
        #   c. Применить temperature scaling: logits = logits / temperature
        #   d. Применить top-k фильтрацию (если задан top_k)
        #   e. Применить top-p (nucleus) фильтрацию (если задан top_p)
        #   f. Вычислить вероятности через softmax
        #   g. Сэмплировать следующий токен (greedy или sampling)
        #   h. Добавить токен к последовательности
        #   i. Обновить attention mask для новой длины
        #   j. Обновить key-value cache (если используется)
        
        # TODO: Обработка критериев остановки
        # - Проверить специальные токены окончания (если есть)
        # - Обрезать до максимальной длины
        # - Вернуть финальную последовательность
        
        # TODO: Вопросы для размышления:
        # - Как эффективно обновлять attention mask при росте последовательности?
        # - Какой формат должен иметь key-value cache для MoE блоков?
        # - Как обрабатывать batch с разными длинами последовательностей?
        # - Как оптимизировать memory usage для длинных последовательностей?
        
        # pass

        # Инициализация переменных для генерации
        generated_ids  = input_ids.clone()
        current_length = input_ids.shape[1]
        # Создаем тензор такой же размерности как input_ids, но заполненный единицами
        attention_mask = torch.ones_like(input_ids)

        while current_length < max_length:
            # a. Forward pass: получить logits для всей последовательности
            logits, _ = self.forward(generated_ids, attention_mask)

            # b. Извлечь logits только для последней позиции (shape: [batch, vocab_size])
            logits = logits[:, -1, :]

            # c. Применить temperature scaling
            # -------------------------------------------------------------
            # Это стандартная формула в LLM!
            # Математика: temperature "сжимает" или "растягивает" logits

            # Temperature > 1: "разогревает" распределение
            # - Более равномерные вероятности
            # - Больше случайности в генерации

            # Temperature < 1: "охлаждает" распределение
            # - Более острые пики вероятностей
            # - Более детерминированная генерация

            # Temperature = 1: без изменений (стандартный softmax)
            probabilities = torch.softmax(logits / temperature, dim=-1)
            # -------------------------------------------------------------

            # d. Применить top-k фильтрацию (если задан)
            # e. Применить top-p фильтрацию (если задан)
            # -------------------------------------------------------------
            # TOP-K алгоритм:
            # 1. Найти k самых вероятных токенов
            # 2. Обнулить ВСЕ остальные вероятности
            # 3. Оставить только top-k токенов

            # TOP-P (nucleus) алгоритм:
            # 1. Отсортировать токены по убыванию вероятности
            # 2. Накапливать вероятности до достижения порога p
            # 3. Обнулить все токены после этого порога

            # Пример:
            # probabilities = [0.4, 0.3, 0.2, 0.1]
            # top_k=2:   [0.4, 0.3, 0.0, 0.0]  # только 2 лучших
            # top_p=0.6: [0.4, 0.3, 0.0, 0.0]  # накопили до 0.7 > 0.6
            if top_k is not None:
                top_k_values, top_k_indices = torch.topk(probabilities, top_k)
                probabilities_filtered = torch.zeros_like(probabilities)
                probabilities_filtered.scatter_(dim=-1, index=top_k_indices, src=top_k_values)
                probabilities = probabilities_filtered

            if top_p is not None:
                # Сортируем вероятности по убыванию
                sorted_probs, sorted_indices = torch.sort(probabilities, descending=True, dim=-1)
                # Вычисляем накопленную сумму
                cumulative_probs = torch.cumsum(sorted_probs, dim=-1)
                # Находим токены, которые нужно удалить (cumsum > top_p)
                # Сдвигаем на 1 вправо, чтобы сохранить хотя бы первый токен
                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
                sorted_indices_to_remove[..., 0] = False
                # Обнуляем вероятности для удаляемых токенов
                sorted_probs[sorted_indices_to_remove] = 0.0
                # Возвращаем вероятности в исходный порядок
                probabilities = torch.zeros_like(probabilities)
                probabilities.scatter_(dim=-1, index=sorted_indices, src=sorted_probs)
            # -------------------------------------------------------------

            # Ре-нормализуем вероятности после фильтрации
            probabilities = probabilities / probabilities.sum(dim=-1, keepdim=True)

            # g. Сэмплировать следующий токен (greedy или sampling)
            if do_sample:
                # Стохастическое сэмплирование из распределения
                next_token = torch.multinomial(probabilities, num_samples=1)
            else:
                # Greedy decoding: выбираем токен с максимальной вероятностью
                next_token = torch.argmax(probabilities, dim=-1, keepdim=True)

            # h. Добавить токен к последовательности
            generated_ids = torch.cat([generated_ids, next_token], dim=-1)

            # i. Обновить attention mask для новой длины
            attention_mask = torch.cat(
                [attention_mask, torch.ones_like(next_token)],
                dim=-1
            )

            # j. Обновить длину последовательности
            current_length += 1

        # Обработка критериев остановки и возврат результата
        return generated_ids


    def chat(
        self,
        prompt: str,
        max_length: int = 100,
        temperature: float = 1.0,
        top_k: Optional[int] = None,
        top_p: Optional[float] = None,
        do_sample: bool = True,
    ) -> str:
        """
        Description:
        ---------------
            Высокоуровневый интерфейс text-to-text.

        Pipeline:
        ---------
        1. Encode: prompt (str) → token_ids (tensor)
        2. Generate: token_ids → generated_ids (автогрессивная генерация)
        3. Decode: generated_ids → response (str)

        Args:
        -----
            prompt: Входной текст от пользователя
            max_length: Максимальная длина сгенерированного текста (в токенах)
            temperature: Температура для сэмплирования (по умолчанию 1.0)
            top_k: Количество токенов для top-k фильтрации (опционально)
            top_p: Порог для nucleus sampling (опционально)
            do_sample: True = стохастическое сэмплирование, False = greedy decoding

        Returns:
        --------
            response: Сгенерированный текст (включая исходный prompt)

        Examples:
        ---------
            >>> # Greedy decoding (детерминированный)
            >>> response = model.chat("Once upon a time", do_sample=False)
            >>> print(response)
            "Once upon a time there was a kingdom..."

            >>> # Nucleus sampling (более креативный)
            >>> response = model.chat("Hello world", temperature=0.8, top_p=0.9)
            >>> print(response)
            "Hello world! How are you doing today?"

            >>> # Top-k sampling
            >>> response = model.chat("The quick brown", temperature=1.0, top_k=40)
            >>> print(response)
            "The quick brown fox jumps over the lazy dog"
        """
        # TODO: Реализуйте три шага: Encode → Generate → Decode
        # TODO: Используйте self.tokenizer для encode/decode
        # TODO: Используйте self.generate() для генерации
        # TODO: Верните сгенерированный текст

        # Вопросы для размышления:
        # - Почему важно использовать тот же tokenizer, что и при обучении?
        # - Как обработать специальные токены (BOS/EOS/PAD) при encode/decode?
        # - Как гарантировать, что сгенерированный текст не превышает max_length?


        # Шаг 1: Encode - преобразование текста в token IDs
        # return_tensors="pt" возвращает PyTorch тензоры
        # add_special_tokens=True добавляет специальные токены (BOS/EOS если есть)
        input_ids = self.tokenizer.encode(
            prompt,
            return_tensors="pt",
            add_special_tokens=True
        )

        # Перемещаем на то же устройство, где находится модель
        # Проверяем device через параметры модели (например, embed_tokens.weight)
        device = next(self.parameters()).device
        input_ids = input_ids.to(device)

        # Шаг 2: Generate - автогрессивная генерация через self.generate()
        generated_ids = self.generate(
            input_ids=input_ids,
            max_length=max_length,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            do_sample=do_sample
        )

        # Шаг 3: Decode - преобразование token IDs обратно в текст
        # skip_special_tokens=True удаляет специальные токены (BOS/EOS/PAD)
        # Берём первую (и единственную) последовательность из batch: generated_ids[0]
        response = self.tokenizer.decode(
            generated_ids[0],
            skip_special_tokens=True
        )

        return response
